# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-8d34e4f344-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# History Workflow Series ("Euro explorative search") 

Consider the following exploratory information need:

> investigate the Euro currency, which are the countries that currently use the Euro and which were their currencies before the Euro. Take a look also to other currencies and find which are the most used.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | occupation    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q38`| Italy   | node |





Also consider

```
wd:Q38 ?p ?obj .
```

is the BGP to retrieve all **properties of Italy**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.



The workload should:


1. Identify the BGP for currency

2. Return the countries which currently use Euro

3. Return the countries belonging to the European Union, with a currency different from the Euro (return also their currency).

4. For each country that now uses Euro return which was the currencies used before

5. Return all types of dollar in the world (US Dollar, Australian Dollar, ecc...)

6. Return the currencies used in at least 2 different countries (return also the number of countries using each currency).

7. Consider the currencies obtained in task 6, for each currency return the number of people using that currency (hint: consider the maximum popolation available for each country).

## Task 1
Identify the BGP for currency

In [2]:
# return all properties for Italy
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q38 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1081'), ('name', 'Human Development Index')]
[('p', 'http://www.wikidata.org/prop/direct/P1082'), ('name', 'population')]
[('p', 'http://www.wikidata.org/prop/direct/P1120'), ('name', 'number of deaths')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1198'), ('name', 'unemployment rate')]
[('p', 'http://www.wikidata.org/prop/direct/P122'), ('name', 'basic form of government')]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalan

In [2]:
# return all properties for Italy
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q38 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 500
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1081'), ('name', 'Human Development Index')]
[('p', 'http://www.wikidata.org/prop/direct/P1082'), ('name', 'population')]
[('p', 'http://www.wikidata.org/prop/direct/P1120'), ('name', 'number of deaths')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1198'), ('name', 'unemployment rate')]
[('p', 'http://www.wikidata.org/prop/direct/P122'), ('name', 'basic form of government')]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalan

In [10]:
#return currency for Italy
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   wd:Q38 wdt:P38 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q4916'), ('name', 'euro')]
1


In [4]:
#return what euro is instance of
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   wd:Q4916 wdt:P31 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q28795800'), ('name', 'decimal currency')]
[('obj', 'http://www.wikidata.org/entity/Q8142'), ('name', 'currency')]
2


Final query for this task

In [3]:
# write your final query
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   wd:Q4916 wdt:P31 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q28795800'), ('name', 'decimal currency')]
[('obj', 'http://www.wikidata.org/entity/Q8142'), ('name', 'currency')]
2


## Task 2
Return the countries which currently use Euro

In [5]:
# return countries with link currency to Euro
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   ?s wdt:P38 wd:Q4916 .
   # get the label
   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q1628531'), ('name', 'Quintanar de la Orden')]
[('s', 'http://www.wikidata.org/entity/Q673727'), ('name', 'Neudau')]
[('s', 'http://www.wikidata.org/entity/Q28124876'), ('name', 'Ilmspan')]
[('s', 'http://www.wikidata.org/entity/Q11959'), ('name', 'Alicante')]
[('s', 'http://www.wikidata.org/entity/Q25362'), ('name', 'Saint Barthélemy')]
[('s', 'http://www.wikidata.org/entity/Q17054'), ('name', 'Martinique')]
[('s', 'http://www.wikidata.org/entity/Q656699'), ('name', 'Montsoreau')]
[('s', 'http://www.wikidata.org/entity/Q624511'), ('name', 'Aracena')]
[('s', 'http://www.wikidata.org/entity/Q1246'), ('name', 'Kosovo')]
[('s', 'http://www.wikidata.org/entity/Q126125'), ('name', 'Saint Martin (French part)')]
[('s', 'http://www.wikidata.org/entity/Q129003'), ('name', 'French Southern and Antarctic Lands')]
[('s', 'http://www.wikidata.org/entity/Q13116'), ('name', 'Benelux')]
[('s', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
[

In [7]:
# return nodes which have Italy linked by instance of
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q38 wdt:P31 ?o.
   # get the label
   ?o sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q3624078'), ('name', 'sovereign state')]
[('o', 'http://www.wikidata.org/entity/Q6256'), ('name', 'country')]
2


In [8]:
## return nodes instance of country with link currency to Euro
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   ?s wdt:P31 wd:Q6256;
      wdt:P38 wd:Q4916.
   # get the label
   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
[('s', 'http://www.wikidata.org/entity/Q211'), ('name', 'Latvia')]
[('s', 'http://www.wikidata.org/entity/Q214'), ('name', 'Slovakia')]
[('s', 'http://www.wikidata.org/entity/Q228'), ('name', 'Andorra')]
[('s', 'http://www.wikidata.org/entity/Q229'), ('name', 'Cyprus')]
[('s', 'http://www.wikidata.org/entity/Q233'), ('name', 'Malta')]
[('s', 'http://www.wikidata.org/entity/Q236'), ('name', 'Montenegro')]
[('s', 'http://www.wikidata.org/entity/Q27'), ('name', 'Ireland')]
[('s', 'http://www.wikidata.org/entity/Q29999'), ('name', 'Kingdom of the Netherlands')]
[('s', 'http://www.wikidata.org/entity/Q33'), ('name', 'Finland')]
[('s', 'http://www.wikidata.org/entity/Q38'), ('name', 'Italy')]
[('s', 'http://www.wikidata.org/entity/Q40'), ('name', 'Austria')]
[('s', 'http://www.wikidata.org/entity/Q41'), ('name', 'Greece')]
[('s', 'http://www.wikidata.org/entity/Q45'), ('name', 'Portugal')]
[('s', 'http://www.wikidata.

In [24]:
#return subproperties of currency
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   ?s wdt:P1647 wdt:P38.
   # get the label
   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [11]:
#return properties for country (wrong variable)
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q6256 ?p ?o.
   # get the label
   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('name', 'software version identifier')]
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('name', 'tabular software version')]
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('name', 'has edition or translation')]
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('name', 'Fonsi Nieto')]
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('name', "Al Stewart's albums in chronological order")]
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('name', 'Category:Adelina Lopes Vieira')]
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('name', 'Lubomír Stoklásek')]
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('name', 'Ciudad Vieja')]
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('name', 'Echániz')]
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('name', 'Natuna')]
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('

In [3]:
#return properties for country 
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q6256 ?p ?o.
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('name', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1051'), ('name', 'PSH ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1282'), ('name', 'OpenStreetMap tag or key')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P150'), ('name', 'contains administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('name', 'Wikidata property')]
[('p', 'http://www.wikidata.org/prop/direct/P1709'), ('name', 'equi

In [12]:
#return nodes subclass of country
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   ?s wdt:P279  wd:Q6256.
   # get the label
   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q28864179'), ('name', 'proposed country')]
[('s', 'http://www.wikidata.org/entity/Q2281140'), ('name', 'safe country of origin')]
[('s', 'http://www.wikidata.org/entity/Q188800'), ('name', 'personal union')]
[('s', 'http://www.wikidata.org/entity/Q1323642'), ('name', 'transcontinental country')]
[('s', 'http://www.wikidata.org/entity/Q132453'), ('name', 'developed country')]
[('s', 'http://www.wikidata.org/entity/Q177323'), ('name', 'developing country')]
[('s', 'http://www.wikidata.org/entity/Q5170193'), ('name', 'core country')]
[('s', 'http://www.wikidata.org/entity/Q63441421'), ('name', 'high-income country')]
[('s', 'http://www.wikidata.org/entity/Q160381'), ('name', 'Western world')]
[('s', 'http://www.wikidata.org/entity/Q112099'), ('name', 'island nation')]
[('s', 'http://www.wikidata.org/entity/Q123480'), ('name', 'landlocked country')]
[('s', 'http://www.wikidata.org/entity/Q3624078'), ('name', 'sovereign state')]
[('s', 'http://

In [13]:
# return nodes which have Kingdom of the Netherlands linked by instance of
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q29999 wdt:P31 ?o.
   # get the label
   ?o sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q20181813'), ('name', 'colonial power')]
[('o', 'http://www.wikidata.org/entity/Q3624078'), ('name', 'sovereign state')]
[('o', 'http://www.wikidata.org/entity/Q6256'), ('name', 'country')]
[('o', 'http://www.wikidata.org/entity/Q417175'), ('name', 'kingdom')]
4


In [14]:
#return all the properties linking Kingdom of the Netherlands and Netherlands
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q29999 ?p wd:Q55.
   # get the label
   ?p sc:name ?name.
}
LIMIT 500
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P150'), ('name', 'contains administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
2


In [98]:
# return nodes which have Netherlands linked by instance of
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q55 wdt:P31 ?o.
   # get the label
   ?o sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q6256'), ('name', 'country')]
[('o', 'http://www.wikidata.org/entity/Q15304003'), ('name', 'country of the Kingdom of the Netherlands')]
2


In [6]:
# return nodes with link country to country and link currency to Euro (some countries are missing like Germany, Spain)
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   ?s wdt:P17 wd:Q6256;
      wdt:P38 wd:Q4916.
   # get the label
   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [5]:
#return objects having subject Italy and property country
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q38 wdt:P17 ?o.
   # get the label
   ?o sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q38'), ('name', 'Italy')]
1


In [5]:
#return nodes which have country Italy
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   ?s wdt:P17 wd:Q38.
   # get the label
   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q109576'), ('name', 'Aegadian Islands')]
[('s', 'http://www.wikidata.org/entity/Q3800298'), ('name', '1992 Italian Open')]
[('s', 'http://www.wikidata.org/entity/Q3857666'), ('name', '1990 Stella Artois Indoor')]
[('s', 'http://www.wikidata.org/entity/Q48851233'), ('name', 'Algeria–Italy relations')]
[('s', 'http://www.wikidata.org/entity/Q7920527'), ('name', 'Venues of the 2006 Winter Olympics')]
[('s', 'http://www.wikidata.org/entity/Q895799'), ('name', '2009 World Amateur Boxing Championships')]
[('s', 'http://www.wikidata.org/entity/Q923273'), ('name', '2005 UEFA European Under-17 Football Championship')]
[('s', 'http://www.wikidata.org/entity/Q47176'), ('name', 'Lombard cuisine')]
[('s', 'http://www.wikidata.org/entity/Q4850939'), ('name', 'Years of Lead')]
[('s', 'http://www.wikidata.org/entity/Q5258020'), ('name', 'Denmark–Italy relations')]
[('s', 'http://www.wikidata.org/entity/Q5354766'), ('name', 'elections in Molise')]
[('s', '

In [7]:
# return nodes instances of country 
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   ?s wdt:P31 wd:Q6256.
   # get the label
   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q16644'), ('name', 'Northern Mariana Islands')]
[('s', 'http://www.wikidata.org/entity/Q207991'), ('name', 'Pahlavi Dynasty')]
[('s', 'http://www.wikidata.org/entity/Q756617'), ('name', 'Danish Realm')]
[('s', 'http://www.wikidata.org/entity/Q786'), ('name', 'Dominican Republic')]
[('s', 'http://www.wikidata.org/entity/Q907112'), ('name', 'Transnistria')]
[('s', 'http://www.wikidata.org/entity/Q244165'), ('name', 'Republic of Artsakh')]
[('s', 'http://www.wikidata.org/entity/Q1005'), ('name', 'The Gambia')]
[('s', 'http://www.wikidata.org/entity/Q1006'), ('name', 'Guinea')]
[('s', 'http://www.wikidata.org/entity/Q1007'), ('name', 'Guinea-Bissau')]
[('s', 'http://www.wikidata.org/entity/Q1008'), ('name', 'Ivory Coast')]
[('s', 'http://www.wikidata.org/entity/Q1009'), ('name', 'Cameroon')]
[('s', 'http://www.wikidata.org/entity/Q1011'), ('name', 'Cape Verde')]
[('s', 'http://www.wikidata.org/entity/Q1013'), ('name', 'Lesotho')]
[('s', 'http:

In [3]:
# return nodes which have Germany linked by instance of
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q183 wdt:P31 ?o.
   # get the label
   ?o sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q3624078'), ('name', 'sovereign state')]
[('o', 'http://www.wikidata.org/entity/Q4209223'), ('name', 'Rechtsstaat')]
[('o', 'http://www.wikidata.org/entity/Q43702'), ('name', 'federation')]
[('o', 'http://www.wikidata.org/entity/Q619610'), ('name', 'social state')]
[('o', 'http://www.wikidata.org/entity/Q63791824'), ('name', 'country bordering the Baltic Sea')]
[('o', 'http://www.wikidata.org/entity/Q7270'), ('name', 'republic')]
6


In [2]:
#same for Spain
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q29 wdt:P31 ?o.
   # get the label
   ?o sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q3624078'), ('name', 'sovereign state')]
1


In [4]:
#return all sublcasses of country (wrong Q is missing in Q6256)
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   ?s wdt:P279 wd:6256.
   # get the label
   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [7]:
#return all sublcasses of country 
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   ?s wdt:P279 wd:Q6256.
   # get the label
   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q28864179'), ('name', 'proposed country')]
[('s', 'http://www.wikidata.org/entity/Q2281140'), ('name', 'safe country of origin')]
[('s', 'http://www.wikidata.org/entity/Q188800'), ('name', 'personal union')]
[('s', 'http://www.wikidata.org/entity/Q1323642'), ('name', 'transcontinental country')]
[('s', 'http://www.wikidata.org/entity/Q132453'), ('name', 'developed country')]
[('s', 'http://www.wikidata.org/entity/Q177323'), ('name', 'developing country')]
[('s', 'http://www.wikidata.org/entity/Q5170193'), ('name', 'core country')]
[('s', 'http://www.wikidata.org/entity/Q63441421'), ('name', 'high-income country')]
[('s', 'http://www.wikidata.org/entity/Q160381'), ('name', 'Western world')]
[('s', 'http://www.wikidata.org/entity/Q112099'), ('name', 'island nation')]
[('s', 'http://www.wikidata.org/entity/Q123480'), ('name', 'landlocked country')]
[('s', 'http://www.wikidata.org/entity/Q3624078'), ('name', 'sovereign state')]
[('s', 'http://

In [51]:
#return all sublcasses of sovereign state
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   ?s wdt:P279 wd:Q3624078.
   # get the label
   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q160016'), ('name', 'member states of the United Nations')]
[('s', 'http://www.wikidata.org/entity/Q189062'), ('name', 'states with nuclear weapons')]
[('s', 'http://www.wikidata.org/entity/Q20181813'), ('name', 'colonial power')]
[('s', 'http://www.wikidata.org/entity/Q48349'), ('name', 'empire')]
[('s', 'http://www.wikidata.org/entity/Q51576574'), ('name', 'Mediterranean country')]
[('s', 'http://www.wikidata.org/entity/Q63791824'), ('name', 'country bordering the Baltic Sea')]
[('s', 'http://www.wikidata.org/entity/Q7840353'), ('name', 'federally recognized Native American tribe in the United States')]
[('s', 'http://www.wikidata.org/entity/Q244381'), ('name', 'microstate')]
[('s', 'http://www.wikidata.org/entity/Q202686'), ('name', 'Commonwealth realm')]
[('s', 'http://www.wikidata.org/entity/Q179671'), ('name', 'nation state')]
[('s', 'http://www.wikidata.org/entity/Q25449457'), ('name', 'continental country')]
[('s', 'http://www.wiki

In [8]:
#return instances of country bordering the Baltic Sea
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   ?s wdt:P31 wd:Q63791824.
   # get the label
   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany')]
[('s', 'http://www.wikidata.org/entity/Q211'), ('name', 'Latvia')]
[('s', 'http://www.wikidata.org/entity/Q33'), ('name', 'Finland')]
[('s', 'http://www.wikidata.org/entity/Q35'), ('name', 'Denmark')]
4


In [13]:
#return what country is instance of
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q6256 wdt:P31 ?o.
   # get the label
   ?o sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q104098715'), ('name', 'territorial entity type')]
1


In [14]:
#return superclass for country
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q6256 wdt:P279 ?o.
   # get the label
   ?o sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q1048835'), ('name', 'political territorial entity')]
1


In [3]:
# return nodes instance of country with link currency to Euro
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   ?s wdt:P31 wd:Q6256;
      wdt:P38 wd:Q4916.
   # get the label
   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
[('s', 'http://www.wikidata.org/entity/Q211'), ('name', 'Latvia')]
[('s', 'http://www.wikidata.org/entity/Q214'), ('name', 'Slovakia')]
[('s', 'http://www.wikidata.org/entity/Q228'), ('name', 'Andorra')]
[('s', 'http://www.wikidata.org/entity/Q229'), ('name', 'Cyprus')]
[('s', 'http://www.wikidata.org/entity/Q233'), ('name', 'Malta')]
[('s', 'http://www.wikidata.org/entity/Q236'), ('name', 'Montenegro')]
[('s', 'http://www.wikidata.org/entity/Q27'), ('name', 'Ireland')]
[('s', 'http://www.wikidata.org/entity/Q29999'), ('name', 'Kingdom of the Netherlands')]
[('s', 'http://www.wikidata.org/entity/Q33'), ('name', 'Finland')]
[('s', 'http://www.wikidata.org/entity/Q38'), ('name', 'Italy')]
[('s', 'http://www.wikidata.org/entity/Q40'), ('name', 'Austria')]
[('s', 'http://www.wikidata.org/entity/Q41'), ('name', 'Greece')]
[('s', 'http://www.wikidata.org/entity/Q45'), ('name', 'Portugal')]
[('s', 'http://www.wikidata.

In [17]:
#Return instances of country or sovereign state which currently use Euro
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   ?s wdt:P31 ?class;
      wdt:P38 wd:Q4916.
   # get the label
   ?s sc:name ?name.
   FILTER(?class IN (wd:Q6256, wd:Q3624078))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
[('s', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany')]
[('s', 'http://www.wikidata.org/entity/Q191'), ('name', 'Estonia')]
[('s', 'http://www.wikidata.org/entity/Q211'), ('name', 'Latvia')]
[('s', 'http://www.wikidata.org/entity/Q214'), ('name', 'Slovakia')]
[('s', 'http://www.wikidata.org/entity/Q215'), ('name', 'Slovenia')]
[('s', 'http://www.wikidata.org/entity/Q228'), ('name', 'Andorra')]
[('s', 'http://www.wikidata.org/entity/Q229'), ('name', 'Cyprus')]
[('s', 'http://www.wikidata.org/entity/Q233'), ('name', 'Malta')]
[('s', 'http://www.wikidata.org/entity/Q235'), ('name', 'Monaco')]
[('s', 'http://www.wikidata.org/entity/Q236'), ('name', 'Montenegro')]
[('s', 'http://www.wikidata.org/entity/Q237'), ('name', 'Vatican City')]
[('s', 'http://www.wikidata.org/entity/Q238'), ('name', 'San Marino')]
[('s', 'http://www.wikidata.org/entity/Q27'), ('name', 'Ireland')]
[('s', 'http://www.wikidata.org/en

In [34]:
#Return instances of a subclass of country which currently use Euro
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   ?s wdt:P31 ?class;
      wdt:P38 wd:Q4916.
   ?class wdt:P279 wd:Q6256.
   # get the label
   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
[('s', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany')]
[('s', 'http://www.wikidata.org/entity/Q191'), ('name', 'Estonia')]
[('s', 'http://www.wikidata.org/entity/Q211'), ('name', 'Latvia')]
[('s', 'http://www.wikidata.org/entity/Q214'), ('name', 'Slovakia')]
[('s', 'http://www.wikidata.org/entity/Q215'), ('name', 'Slovenia')]
[('s', 'http://www.wikidata.org/entity/Q228'), ('name', 'Andorra')]
[('s', 'http://www.wikidata.org/entity/Q229'), ('name', 'Cyprus')]
[('s', 'http://www.wikidata.org/entity/Q233'), ('name', 'Malta')]
[('s', 'http://www.wikidata.org/entity/Q235'), ('name', 'Monaco')]
[('s', 'http://www.wikidata.org/entity/Q236'), ('name', 'Montenegro')]
[('s', 'http://www.wikidata.org/entity/Q237'), ('name', 'Vatican City')]
[('s', 'http://www.wikidata.org/entity/Q238'), ('name', 'San Marino')]
[('s', 'http://www.wikidata.org/entity/Q27'), ('name', 'Ireland')]
[('s', 'http://www.wikidata.org/en

In [18]:
#Return subjects linked to Euro by currency property
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   ?s wdt:P38 wd:Q4916.
   # get the label
   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q1628531'), ('name', 'Quintanar de la Orden')]
[('s', 'http://www.wikidata.org/entity/Q673727'), ('name', 'Neudau')]
[('s', 'http://www.wikidata.org/entity/Q28124876'), ('name', 'Ilmspan')]
[('s', 'http://www.wikidata.org/entity/Q11959'), ('name', 'Alicante')]
[('s', 'http://www.wikidata.org/entity/Q25362'), ('name', 'Saint Barthélemy')]
[('s', 'http://www.wikidata.org/entity/Q17054'), ('name', 'Martinique')]
[('s', 'http://www.wikidata.org/entity/Q656699'), ('name', 'Montsoreau')]
[('s', 'http://www.wikidata.org/entity/Q624511'), ('name', 'Aracena')]
[('s', 'http://www.wikidata.org/entity/Q1246'), ('name', 'Kosovo')]
[('s', 'http://www.wikidata.org/entity/Q126125'), ('name', 'Saint Martin (French part)')]
[('s', 'http://www.wikidata.org/entity/Q129003'), ('name', 'French Southern and Antarctic Lands')]
[('s', 'http://www.wikidata.org/entity/Q13116'), ('name', 'Benelux')]
[('s', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
[

In [9]:
#Return the countries which currently use Euro filter country instance of country or sovereign state
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   ?s wdt:P31 ?class;
      wdt:P38 wd:Q4916.
   # get the label
   ?s sc:name ?name.
   FILTER(?class IN (wd:Q6256, wd:Q3624078))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
[('s', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany')]
[('s', 'http://www.wikidata.org/entity/Q191'), ('name', 'Estonia')]
[('s', 'http://www.wikidata.org/entity/Q211'), ('name', 'Latvia')]
[('s', 'http://www.wikidata.org/entity/Q214'), ('name', 'Slovakia')]
[('s', 'http://www.wikidata.org/entity/Q215'), ('name', 'Slovenia')]
[('s', 'http://www.wikidata.org/entity/Q228'), ('name', 'Andorra')]
[('s', 'http://www.wikidata.org/entity/Q229'), ('name', 'Cyprus')]
[('s', 'http://www.wikidata.org/entity/Q233'), ('name', 'Malta')]
[('s', 'http://www.wikidata.org/entity/Q235'), ('name', 'Monaco')]
[('s', 'http://www.wikidata.org/entity/Q236'), ('name', 'Montenegro')]
[('s', 'http://www.wikidata.org/entity/Q237'), ('name', 'Vatican City')]
[('s', 'http://www.wikidata.org/entity/Q238'), ('name', 'San Marino')]
[('s', 'http://www.wikidata.org/entity/Q27'), ('name', 'Ireland')]
[('s', 'http://www.wikidata.org/en

In [4]:
#Return the countries which currently use Euro with * operator
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   ?s wdt:P31 ?class;
      wdt:P38 wd:Q4916.
   ?class wdt:P279* wd:Q6256.
   # get the label
   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
[('s', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany')]
[('s', 'http://www.wikidata.org/entity/Q191'), ('name', 'Estonia')]
[('s', 'http://www.wikidata.org/entity/Q211'), ('name', 'Latvia')]
[('s', 'http://www.wikidata.org/entity/Q214'), ('name', 'Slovakia')]
[('s', 'http://www.wikidata.org/entity/Q215'), ('name', 'Slovenia')]
[('s', 'http://www.wikidata.org/entity/Q228'), ('name', 'Andorra')]
[('s', 'http://www.wikidata.org/entity/Q229'), ('name', 'Cyprus')]
[('s', 'http://www.wikidata.org/entity/Q233'), ('name', 'Malta')]
[('s', 'http://www.wikidata.org/entity/Q235'), ('name', 'Monaco')]
[('s', 'http://www.wikidata.org/entity/Q236'), ('name', 'Montenegro')]
[('s', 'http://www.wikidata.org/entity/Q237'), ('name', 'Vatican City')]
[('s', 'http://www.wikidata.org/entity/Q238'), ('name', 'San Marino')]
[('s', 'http://www.wikidata.org/entity/Q27'), ('name', 'Ireland')]
[('s', 'http://www.wikidata.org/en

In [6]:
#Return the countries which currently use Euro with + operator
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   ?s wdt:P31 ?class;
      wdt:P38 wd:Q4916.
   ?class wdt:P279+ wd:Q6256.
   # get the label
   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
[('s', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany')]
[('s', 'http://www.wikidata.org/entity/Q191'), ('name', 'Estonia')]
[('s', 'http://www.wikidata.org/entity/Q211'), ('name', 'Latvia')]
[('s', 'http://www.wikidata.org/entity/Q214'), ('name', 'Slovakia')]
[('s', 'http://www.wikidata.org/entity/Q215'), ('name', 'Slovenia')]
[('s', 'http://www.wikidata.org/entity/Q228'), ('name', 'Andorra')]
[('s', 'http://www.wikidata.org/entity/Q229'), ('name', 'Cyprus')]
[('s', 'http://www.wikidata.org/entity/Q233'), ('name', 'Malta')]
[('s', 'http://www.wikidata.org/entity/Q235'), ('name', 'Monaco')]
[('s', 'http://www.wikidata.org/entity/Q236'), ('name', 'Montenegro')]
[('s', 'http://www.wikidata.org/entity/Q237'), ('name', 'Vatican City')]
[('s', 'http://www.wikidata.org/entity/Q238'), ('name', 'San Marino')]
[('s', 'http://www.wikidata.org/entity/Q27'), ('name', 'Ireland')]
[('s', 'http://www.wikidata.org/en

Final query for this task

In [4]:
# write your final query
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   ?s wdt:P31/wdt:P279* wd:Q6256;
      wdt:P38 wd:Q4916.

   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
[('s', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany')]
[('s', 'http://www.wikidata.org/entity/Q191'), ('name', 'Estonia')]
[('s', 'http://www.wikidata.org/entity/Q211'), ('name', 'Latvia')]
[('s', 'http://www.wikidata.org/entity/Q214'), ('name', 'Slovakia')]
[('s', 'http://www.wikidata.org/entity/Q215'), ('name', 'Slovenia')]
[('s', 'http://www.wikidata.org/entity/Q228'), ('name', 'Andorra')]
[('s', 'http://www.wikidata.org/entity/Q229'), ('name', 'Cyprus')]
[('s', 'http://www.wikidata.org/entity/Q233'), ('name', 'Malta')]
[('s', 'http://www.wikidata.org/entity/Q235'), ('name', 'Monaco')]
[('s', 'http://www.wikidata.org/entity/Q236'), ('name', 'Montenegro')]
[('s', 'http://www.wikidata.org/entity/Q237'), ('name', 'Vatican City')]
[('s', 'http://www.wikidata.org/entity/Q238'), ('name', 'San Marino')]
[('s', 'http://www.wikidata.org/entity/Q27'), ('name', 'Ireland')]
[('s', 'http://www.wikidata.org/en

## Task 3
Return the countries belonging to the European Union, with a currency different from the Euro (return also their currency).

In [37]:
# return what Italy is member of (wrong variable)
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q38 wdt:P463 ?o.
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q488981'), ('name', 'software version identifier')]
[('o', 'http://www.wikidata.org/entity/Q488981'), ('name', 'tabular software version')]
[('o', 'http://www.wikidata.org/entity/Q488981'), ('name', 'has edition or translation')]
[('o', 'http://www.wikidata.org/entity/Q488981'), ('name', 'Fonsi Nieto')]
[('o', 'http://www.wikidata.org/entity/Q488981'), ('name', "Al Stewart's albums in chronological order")]
[('o', 'http://www.wikidata.org/entity/Q488981'), ('name', 'Category:Adelina Lopes Vieira')]
[('o', 'http://www.wikidata.org/entity/Q488981'), ('name', 'Lubomír Stoklásek')]
[('o', 'http://www.wikidata.org/entity/Q488981'), ('name', 'Ciudad Vieja')]
[('o', 'http://www.wikidata.org/entity/Q488981'), ('name', 'Echániz')]
[('o', 'http://www.wikidata.org/entity/Q488981'), ('name', 'Natuna')]
[('o', 'http://www.wikidata.org/entity/Q488981'), ('name', "Acid Mothers Temple's albums in chronological order")]
[('o', 'http://www.wikidata.org/enti

In [56]:
# return what Italy is part of
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q38 wdt:P361 ?o.
   # get the label
   ?o sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q46'), ('name', 'Europe')]
1


In [57]:
# return property for Europe
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q46 ?p ?o.
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P7607'), ('name', 'WikiTree category or space')]
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1082'), ('name', 'population')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1281'), ('name', 'WOEID')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', '

In [58]:
# return what is different from Europe
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q46 wdt:P1889 ?o.
   # get the label
   ?o sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q345567'), ('name', 'Europe')]
[('o', 'http://www.wikidata.org/entity/Q741380'), ('name', 'Europa')]
2


In [59]:
# return what Itlay is part of
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q38 wdt:P361 ?o.
   # get the label
   ?o sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q46'), ('name', 'Europe')]
1


In [60]:
# return what Itlay is member of
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q38 wdt:P463 ?o.
   # get the label
   ?o sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q488981'), ('name', 'European Bank for Reconstruction and Development')]
[('o', 'http://www.wikidata.org/entity/Q1153087'), ('name', 'Inter-American Development Bank')]
[('o', 'http://www.wikidata.org/entity/Q848633'), ('name', 'International Chamber of Commerce')]
[('o', 'http://www.wikidata.org/entity/Q1043527'), ('name', 'Multilateral Investment Guarantee Agency')]
[('o', 'http://www.wikidata.org/entity/Q1065'), ('name', 'United Nations')]
[('o', 'http://www.wikidata.org/entity/Q1072120'), ('name', 'Missile Technology Control Regime')]
[('o', 'http://www.wikidata.org/entity/Q123209'), ('name', 'Latin Union')]
[('o', 'http://www.wikidata.org/entity/Q125761'), ('name', 'International Civil Aviation Organization')]
[('o', 'http://www.wikidata.org/entity/Q1377612'), ('name', 'European Payments Union')]
[('o', 'http://www.wikidata.org/entity/Q141720'), ('name', 'European Air Transport Command')]
[('o', 'http://www.wikidata.org/entity/Q148079

In [19]:
# return all property of European Union (wrong code)
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q468 ?p ?o.
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1080'), ('name', 'from narrative universe')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P1441'), ('name', 'present in work')]
[('p', 'http://www.wikidata.org/prop/direct/P170'), ('name', 'creator')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P6262'), ('name', 'Fandom article ID')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('name', 'Freebase ID')]
[('p', 'http://www.wikidata.org/prop/direct/P973'), ('name', 'described at URL')]
9


In [20]:
# return all property of European Union (correct code)
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q458 ?p ?o.
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1051'), ('name', 'PSH ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1081'), ('name', 'Human Development Index')]
[('p', 'http://www.wikidata.org/prop/direct/P1082'), ('name', 'population')]
[('p', 'http://www.wikidata.org/prop/direct/P112'), ('name', 'founded by')]
[('p', 'http://www.wikidata.org/prop/direct/P1125'), ('name', 'Gini coefficient')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1273'), ('name', 'CANTIC ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1280'), ('name', 'CONOR.SI ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1313'), ('nam

In [22]:
# return all property from European Union to Italy
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q458 ?p wd:Q38 .
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P112'), ('name', 'founded by')]
[('p', 'http://www.wikidata.org/prop/direct/P150'), ('name', 'contains administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part')]
3


In [23]:
# return all property from Italy to EU
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q38 ?p wd:Q458 .
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P463'), ('name', 'member of')]
1


In [10]:
# return all countries belonging to EU
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   ?s wdt:P31 wd:Q6256;
      wdt:P463 wd:Q458.

   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
[('s', 'http://www.wikidata.org/entity/Q145'), ('name', 'United Kingdom')]
[('s', 'http://www.wikidata.org/entity/Q211'), ('name', 'Latvia')]
[('s', 'http://www.wikidata.org/entity/Q213'), ('name', 'Czech Republic')]
[('s', 'http://www.wikidata.org/entity/Q214'), ('name', 'Slovakia')]
[('s', 'http://www.wikidata.org/entity/Q218'), ('name', 'Romania')]
[('s', 'http://www.wikidata.org/entity/Q219'), ('name', 'Bulgaria')]
[('s', 'http://www.wikidata.org/entity/Q224'), ('name', 'Croatia')]
[('s', 'http://www.wikidata.org/entity/Q229'), ('name', 'Cyprus')]
[('s', 'http://www.wikidata.org/entity/Q233'), ('name', 'Malta')]
[('s', 'http://www.wikidata.org/entity/Q27'), ('name', 'Ireland')]
[('s', 'http://www.wikidata.org/entity/Q28'), ('name', 'Hungary')]
[('s', 'http://www.wikidata.org/entity/Q29999'), ('name', 'Kingdom of the Netherlands')]
[('s', 'http://www.wikidata.org/entity/Q33'), ('name', 'Finland')]
[('s', 'htt

In [12]:
# return subjects member of EU
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   ?s wdt:P463 wd:Q458.

   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
[('s', 'http://www.wikidata.org/entity/Q145'), ('name', 'United Kingdom')]
[('s', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany')]
[('s', 'http://www.wikidata.org/entity/Q191'), ('name', 'Estonia')]
[('s', 'http://www.wikidata.org/entity/Q211'), ('name', 'Latvia')]
[('s', 'http://www.wikidata.org/entity/Q213'), ('name', 'Czech Republic')]
[('s', 'http://www.wikidata.org/entity/Q214'), ('name', 'Slovakia')]
[('s', 'http://www.wikidata.org/entity/Q215'), ('name', 'Slovenia')]
[('s', 'http://www.wikidata.org/entity/Q218'), ('name', 'Romania')]
[('s', 'http://www.wikidata.org/entity/Q219'), ('name', 'Bulgaria')]
[('s', 'http://www.wikidata.org/entity/Q224'), ('name', 'Croatia')]
[('s', 'http://www.wikidata.org/entity/Q229'), ('name', 'Cyprus')]
[('s', 'http://www.wikidata.org/entity/Q233'), ('name', 'Malta')]
[('s', 'http://www.wikidata.org/entity/Q27'), ('name', 'Ireland')]
[('s', 'http://www.wikidata.o

In [11]:
# return all countries member of EU with currency different from Euro
queryString = """
SELECT DISTINCT ?s ?name 
WHERE {
   ?s wdt:P31 wd:Q6256;
      wdt:P463 wd:Q458;
      wdt:P38 ?currency.
   FILTER(?currency != wd:Q4916)
   
   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q145'), ('name', 'United Kingdom')]
[('s', 'http://www.wikidata.org/entity/Q213'), ('name', 'Czech Republic')]
[('s', 'http://www.wikidata.org/entity/Q218'), ('name', 'Romania')]
[('s', 'http://www.wikidata.org/entity/Q219'), ('name', 'Bulgaria')]
[('s', 'http://www.wikidata.org/entity/Q224'), ('name', 'Croatia')]
[('s', 'http://www.wikidata.org/entity/Q28'), ('name', 'Hungary')]
[('s', 'http://www.wikidata.org/entity/Q34'), ('name', 'Sweden')]
[('s', 'http://www.wikidata.org/entity/Q35'), ('name', 'Denmark')]
[('s', 'http://www.wikidata.org/entity/Q36'), ('name', 'Poland')]
9


In [29]:
# return all coutries member of EU with currency different from Euro and their currency
queryString = """ 
SELECT DISTINCT ?s ?name ?currency ?currencyName
WHERE {
   ?s wdt:P31 wd:Q6256;
      wdt:P463 wd:Q458;
      wdt:P38 ?currency.
   FILTER(?currency != wd:Q4916)
   
   ?s sc:name ?name.
   ?currency sc:name ?currencyName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q36'), ('name', 'Poland'), ('currency', 'http://www.wikidata.org/entity/Q123213'), ('currencyName', 'złoty')]
[('s', 'http://www.wikidata.org/entity/Q219'), ('name', 'Bulgaria'), ('currency', 'http://www.wikidata.org/entity/Q172540'), ('currencyName', 'Bulgarian lev')]
[('s', 'http://www.wikidata.org/entity/Q28'), ('name', 'Hungary'), ('currency', 'http://www.wikidata.org/entity/Q47190'), ('currencyName', 'forint')]
[('s', 'http://www.wikidata.org/entity/Q218'), ('name', 'Romania'), ('currency', 'http://www.wikidata.org/entity/Q131645'), ('currencyName', 'Romanian leu')]
[('s', 'http://www.wikidata.org/entity/Q145'), ('name', 'United Kingdom'), ('currency', 'http://www.wikidata.org/entity/Q25224'), ('currencyName', 'pound sterling')]
[('s', 'http://www.wikidata.org/entity/Q224'), ('name', 'Croatia'), ('currency', 'http://www.wikidata.org/entity/Q26360'), ('currencyName', 'Croatian kuna')]
[('s', 'http://www.wikidata.org/entity/Q213'), ('na

In [5]:
#Return the countries member of the European Union, with a currency different from the Euro (return also their currency).
#using * operator
queryString = """ 
SELECT DISTINCT ?s ?name ?currency ?currencyName
WHERE {
   ?s wdt:P31 ?class;
      wdt:P463 wd:Q458;
      wdt:P38 ?currency.
   ?class wdt:P279* wd:Q6256
   FILTER(?currency != wd:Q4916)
   
   ?s sc:name ?name.
   ?currency sc:name ?currencyName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q145'), ('name', 'United Kingdom'), ('currency', 'http://www.wikidata.org/entity/Q25224'), ('currencyName', 'pound sterling')]
[('s', 'http://www.wikidata.org/entity/Q213'), ('name', 'Czech Republic'), ('currency', 'http://www.wikidata.org/entity/Q131016'), ('currencyName', 'Czech koruna')]
[('s', 'http://www.wikidata.org/entity/Q218'), ('name', 'Romania'), ('currency', 'http://www.wikidata.org/entity/Q131645'), ('currencyName', 'Romanian leu')]
[('s', 'http://www.wikidata.org/entity/Q219'), ('name', 'Bulgaria'), ('currency', 'http://www.wikidata.org/entity/Q172540'), ('currencyName', 'Bulgarian lev')]
[('s', 'http://www.wikidata.org/entity/Q224'), ('name', 'Croatia'), ('currency', 'http://www.wikidata.org/entity/Q26360'), ('currencyName', 'Croatian kuna')]
[('s', 'http://www.wikidata.org/entity/Q28'), ('name', 'Hungary'), ('currency', 'http://www.wikidata.org/entity/Q47190'), ('currencyName', 'forint')]
[('s', 'http://www.wikidata.org/ent

In [9]:
# return all countries member of EU with currency different from Euro and their currency 
#(instance of country with * operator on instance of)
queryString = """ 
SELECT DISTINCT ?s ?name ?currency ?currencyName
WHERE {
   ?s wdt:P31* wd:Q6256;
      wdt:P463 wd:Q458;
      wdt:P38 ?currency.
   FILTER(?currency != wd:Q4916)
   
   ?s sc:name ?name.
   ?currency sc:name ?currencyName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q145'), ('name', 'United Kingdom'), ('currency', 'http://www.wikidata.org/entity/Q25224'), ('currencyName', 'pound sterling')]
[('s', 'http://www.wikidata.org/entity/Q213'), ('name', 'Czech Republic'), ('currency', 'http://www.wikidata.org/entity/Q131016'), ('currencyName', 'Czech koruna')]
[('s', 'http://www.wikidata.org/entity/Q218'), ('name', 'Romania'), ('currency', 'http://www.wikidata.org/entity/Q131645'), ('currencyName', 'Romanian leu')]
[('s', 'http://www.wikidata.org/entity/Q219'), ('name', 'Bulgaria'), ('currency', 'http://www.wikidata.org/entity/Q172540'), ('currencyName', 'Bulgarian lev')]
[('s', 'http://www.wikidata.org/entity/Q224'), ('name', 'Croatia'), ('currency', 'http://www.wikidata.org/entity/Q26360'), ('currencyName', 'Croatian kuna')]
[('s', 'http://www.wikidata.org/entity/Q28'), ('name', 'Hungary'), ('currency', 'http://www.wikidata.org/entity/Q47190'), ('currencyName', 'forint')]
[('s', 'http://www.wikidata.org/ent

In [6]:
#Return the countries (instance of country or a subclass of it) belonging to the European Union, with a currency different from the Euro 
queryString = """ 
SELECT DISTINCT ?s ?name ?currency ?currencyName
WHERE {
   ?s wdt:P31/wdt:P279* wd:Q6256;
      wdt:P463 wd:Q458;
      wdt:P38 ?currency.
   FILTER(?currency != wd:Q4916)
   
   ?s sc:name ?name.
   ?currency sc:name ?currencyName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q145'), ('name', 'United Kingdom'), ('currency', 'http://www.wikidata.org/entity/Q25224'), ('currencyName', 'pound sterling')]
[('s', 'http://www.wikidata.org/entity/Q213'), ('name', 'Czech Republic'), ('currency', 'http://www.wikidata.org/entity/Q131016'), ('currencyName', 'Czech koruna')]
[('s', 'http://www.wikidata.org/entity/Q218'), ('name', 'Romania'), ('currency', 'http://www.wikidata.org/entity/Q131645'), ('currencyName', 'Romanian leu')]
[('s', 'http://www.wikidata.org/entity/Q219'), ('name', 'Bulgaria'), ('currency', 'http://www.wikidata.org/entity/Q172540'), ('currencyName', 'Bulgarian lev')]
[('s', 'http://www.wikidata.org/entity/Q224'), ('name', 'Croatia'), ('currency', 'http://www.wikidata.org/entity/Q26360'), ('currencyName', 'Croatian kuna')]
[('s', 'http://www.wikidata.org/entity/Q28'), ('name', 'Hungary'), ('currency', 'http://www.wikidata.org/entity/Q47190'), ('currencyName', 'forint')]
[('s', 'http://www.wikidata.org/ent

Final query for this task

In [7]:
# write your final query
queryString = """ 
SELECT DISTINCT ?s ?name ?currency ?currencyName
WHERE {
   ?s wdt:P31/wdt:P279* wd:Q6256;
      wdt:P463 wd:Q458;
      wdt:P38 ?currency.
   FILTER(?currency != wd:Q4916)
   
   ?s sc:name ?name.
   ?currency sc:name ?currencyName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q145'), ('name', 'United Kingdom'), ('currency', 'http://www.wikidata.org/entity/Q25224'), ('currencyName', 'pound sterling')]
[('s', 'http://www.wikidata.org/entity/Q213'), ('name', 'Czech Republic'), ('currency', 'http://www.wikidata.org/entity/Q131016'), ('currencyName', 'Czech koruna')]
[('s', 'http://www.wikidata.org/entity/Q218'), ('name', 'Romania'), ('currency', 'http://www.wikidata.org/entity/Q131645'), ('currencyName', 'Romanian leu')]
[('s', 'http://www.wikidata.org/entity/Q219'), ('name', 'Bulgaria'), ('currency', 'http://www.wikidata.org/entity/Q172540'), ('currencyName', 'Bulgarian lev')]
[('s', 'http://www.wikidata.org/entity/Q224'), ('name', 'Croatia'), ('currency', 'http://www.wikidata.org/entity/Q26360'), ('currencyName', 'Croatian kuna')]
[('s', 'http://www.wikidata.org/entity/Q28'), ('name', 'Hungary'), ('currency', 'http://www.wikidata.org/entity/Q47190'), ('currencyName', 'forint')]
[('s', 'http://www.wikidata.org/ent

## Task 4
For each country that now uses Euro return which was the currencies used before

In [3]:
#return all properties for currency
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q8142 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1051'), ('name', 'PSH ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1282'), ('name', 'OpenStreetMap tag or key')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1482'), ('name', 'Stack Exchange tag')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('name', 'Wikidata property')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P2184')

In [8]:
#return superclasses for currency
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   wd:Q8142 wdt:P279 ?obj .
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q11105360'), ('name', 'medium of exchange')]
[('obj', 'http://www.wikidata.org/entity/Q47574'), ('name', 'unit of measurement')]
[('obj', 'http://www.wikidata.org/entity/Q831772'), ('name', 'foreign currencies')]
[('obj', 'http://www.wikidata.org/entity/Q1368'), ('name', 'money')]
4


In [7]:
## return all properties for Greece
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q41 ?p ?o.
   # get the label
   ?p sc:name ?name.
}
LIMIT 300
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1081'), ('name', 'Human Development Index')]
[('p', 'http://www.wikidata.org/prop/direct/P1082'), ('name', 'population')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1198'), ('name', 'unemployment rate')]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1304'), ('name', 'central bank')]
[('p', 'http://www.wikidata.org/prop/direct/P1313'), ('name', 'office held by head of government')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source'

In [9]:
## return currency for Greece
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q41 wdt:P38 ?o.
   # get the label
   ?o sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q4916'), ('name', 'euro')]
1


In [13]:
# return currency for Slovakia
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q214 wdt:P38 ?o.
   # get the label
   ?o sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q4916'), ('name', 'euro')]
1


In [13]:
## return properties for Euro
queryString = """
SELECT ?p ?name
WHERE {
   wd:Q4916 ?p ?o.
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1001'), ('name', 'applies to jurisdiction')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1282'), ('name', 'OpenStreetMap tag or key')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces')]
[('p', 'http://www.wikidata.org/prop/direct/

In [10]:
#return currencies replaced by Euro
queryString = """
SELECT ?obj ?name
WHERE {
   wd:Q4916 wdt:P1365 ?obj.
   # get the label
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q16068'), ('name', 'Deutsche Mark')]
[('obj', 'http://www.wikidata.org/entity/Q788472'), ('name', 'Dutch guilder')]
[('obj', 'http://www.wikidata.org/entity/Q203354'), ('name', 'Finnish markka')]
[('obj', 'http://www.wikidata.org/entity/Q270642'), ('name', 'Irish pound')]
[('obj', 'http://www.wikidata.org/entity/Q184172'), ('name', 'French franc')]
[('obj', 'http://www.wikidata.org/entity/Q189097'), ('name', 'peseta')]
[('obj', 'http://www.wikidata.org/entity/Q204992'), ('name', 'Italian lira')]
[('obj', 'http://www.wikidata.org/entity/Q468474'), ('name', 'Portuguese escudo')]
[('obj', 'http://www.wikidata.org/entity/Q1546443'), ('name', 'Greek drachma')]
[('obj', 'http://www.wikidata.org/entity/Q133011'), ('name', 'Latvian lats')]
[('obj', 'http://www.wikidata.org/entity/Q174467'), ('name', 'Maltese lira')]
[('obj', 'http://www.wikidata.org/entity/Q183354'), ('name', 'Vatican lira')]
[('obj', 'http://www.wikidata.org/entity/Q129983'), (

In [8]:
#return properties for Italian lira
queryString = """
SELECT ?p ?name
WHERE {
   wd:Q204992 ?p ?o.
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1001'), ('name', 'applies to jurisdiction')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces')]
[('p', 'http://www.wikidata.org/prop/direct/P1366'), ('name', 'replaced by')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P176'), ('name', 'manufacturer')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]

In [10]:
#return what Italian lira is instance of
queryString = """
SELECT ?o ?name
WHERE {
   wd:Q204992 wdt:P31 ?o.
   # get the label
   ?o sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q28783456'), ('name', 'obsolete currency')]
[('o', 'http://www.wikidata.org/entity/Q7995739'), ('name', 'lira')]
2


In [12]:
#return country for Italian lira
queryString = """
SELECT ?o ?name
WHERE {
   wd:Q204992 wdt:P17 ?o.
   # get the label
   ?o sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q172579'), ('name', 'Kingdom of Italy')]
[('o', 'http://www.wikidata.org/entity/Q38'), ('name', 'Italy')]
2


In [15]:
#return what italian lira is replaced by
queryString = """
SELECT ?o ?name
WHERE {
   wd:Q204992 wdt:P1366 ?o.
   # get the label
   ?o sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q4916'), ('name', 'euro')]
1


In [15]:
#final query try
queryString = """
SELECT ?country ?cName ?previousCurrency ?currName
WHERE {
   ?country wdt:P31 wd:Q6256;
            wdt:P38 wd:Q4916.
   ?previousCurrency wdt:P1366 wd:Q4916;
                     wdt:P17 ?country.
   
   # get the label
   ?country sc:name ?cName.
   ?previousCurrency sc:name ?currName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q142'), ('cName', 'France'), ('previousCurrency', 'http://www.wikidata.org/entity/Q184172'), ('currName', 'French franc')]
[('country', 'http://www.wikidata.org/entity/Q211'), ('cName', 'Latvia'), ('previousCurrency', 'http://www.wikidata.org/entity/Q133011'), ('currName', 'Latvian lats')]
[('country', 'http://www.wikidata.org/entity/Q214'), ('cName', 'Slovakia'), ('previousCurrency', 'http://www.wikidata.org/entity/Q178874'), ('currName', 'Slovak koruna')]
[('country', 'http://www.wikidata.org/entity/Q228'), ('cName', 'Andorra'), ('previousCurrency', 'http://www.wikidata.org/entity/Q184172'), ('currName', 'French franc')]
[('country', 'http://www.wikidata.org/entity/Q228'), ('cName', 'Andorra'), ('previousCurrency', 'http://www.wikidata.org/entity/Q189097'), ('currName', 'peseta')]
[('country', 'http://www.wikidata.org/entity/Q229'), ('cName', 'Cyprus'), ('previousCurrency', 'http://www.wikidata.org/entity/Q193716'), ('currName', 'C

In [16]:
#return properties linking Italy and Italian lira
queryString = """
SELECT ?p ?name
WHERE {
   wd:Q38 ?p wd:Q204992.
   # get the label
   ?p sc:name ?name
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [15]:
#return properties linking Italian lira and Italy
queryString = """
SELECT ?p ?name
WHERE {
   wd:Q204992 ?p wd:Q38.
   # get the label
   ?p sc:name ?name
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1001'), ('name', 'applies to jurisdiction')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
2


In [17]:
#return country for Italian lira
queryString = """
SELECT ?o ?name
WHERE {
   wd:Q204992 wdt:P17 ?o.
   # get the label
   ?o sc:name ?name
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q172579'), ('name', 'Kingdom of Italy')]
[('o', 'http://www.wikidata.org/entity/Q38'), ('name', 'Italy')]
2


In [16]:
#search for previous currency for Montenegro
queryString = """
SELECT ?country ?cName ?previousCurrency ?currName
WHERE {
   ?previousCurrency wdt:P1366 wd:Q4916;
                     wdt:P17|wdt:P1001 wd:Q236.
   
   # get the label
   ?country sc:name ?cName.
   ?previousCurrency sc:name ?currName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [13]:
#For each country that now uses Euro return which was the currencies used before
queryString = """
SELECT ?country ?cName ?previousCurrency ?currName
WHERE {
   ?country wdt:P31 wd:Q6256;
            wdt:P38 wd:Q4916.
   ?previousCurrency wdt:P1366 wd:Q4916;
                     wdt:P17 ?country.
   
   # get the label
   ?country sc:name ?cName.
   ?previousCurrency sc:name ?currName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q142'), ('cName', 'France'), ('previousCurrency', 'http://www.wikidata.org/entity/Q184172'), ('currName', 'French franc')]
[('country', 'http://www.wikidata.org/entity/Q211'), ('cName', 'Latvia'), ('previousCurrency', 'http://www.wikidata.org/entity/Q133011'), ('currName', 'Latvian lats')]
[('country', 'http://www.wikidata.org/entity/Q214'), ('cName', 'Slovakia'), ('previousCurrency', 'http://www.wikidata.org/entity/Q178874'), ('currName', 'Slovak koruna')]
[('country', 'http://www.wikidata.org/entity/Q228'), ('cName', 'Andorra'), ('previousCurrency', 'http://www.wikidata.org/entity/Q184172'), ('currName', 'French franc')]
[('country', 'http://www.wikidata.org/entity/Q228'), ('cName', 'Andorra'), ('previousCurrency', 'http://www.wikidata.org/entity/Q189097'), ('currName', 'peseta')]
[('country', 'http://www.wikidata.org/entity/Q229'), ('cName', 'Cyprus'), ('previousCurrency', 'http://www.wikidata.org/entity/Q193716'), ('currName', 'C

In [13]:
#For each country (with * operator) that now uses Euro return which was the currencies used before
queryString = """
SELECT DISTINCT ?country ?cName ?previousCurrency ?currName
WHERE {
   ?country wdt:P31 ?class;
            wdt:P38 wd:Q4916.
   ?class wdt:P279* wd:Q6256.
   ?previousCurrency wdt:P1366 wd:Q4916;
                     wdt:P17 ?country.
   
   # get the label
   ?country sc:name ?cName.
   ?previousCurrency sc:name ?currName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q183'), ('cName', 'Germany'), ('previousCurrency', 'http://www.wikidata.org/entity/Q16068'), ('currName', 'Deutsche Mark')]
[('country', 'http://www.wikidata.org/entity/Q29999'), ('cName', 'Kingdom of the Netherlands'), ('previousCurrency', 'http://www.wikidata.org/entity/Q788472'), ('currName', 'Dutch guilder')]
[('country', 'http://www.wikidata.org/entity/Q33'), ('cName', 'Finland'), ('previousCurrency', 'http://www.wikidata.org/entity/Q203354'), ('currName', 'Finnish markka')]
[('country', 'http://www.wikidata.org/entity/Q27'), ('cName', 'Ireland'), ('previousCurrency', 'http://www.wikidata.org/entity/Q270642'), ('currName', 'Irish pound')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('cName', 'France'), ('previousCurrency', 'http://www.wikidata.org/entity/Q184172'), ('currName', 'French franc')]
[('country', 'http://www.wikidata.org/entity/Q228'), ('cName', 'Andorra'), ('previousCurrency', 'http://www.wikidata.org/entity

In [17]:
#For each country (with + operator) that now uses Euro return which was the currencies used before
queryString = """
SELECT DISTINCT ?country ?cName ?previousCurrency ?currName
WHERE {
   ?country wdt:P31 ?class;
            wdt:P38 wd:Q4916.
   ?class wdt:P279+ wd:Q6256.
   ?previousCurrency wdt:P1366 wd:Q4916;
                     wdt:P17 ?country.
   
   # get the label
   ?country sc:name ?cName.
   ?previousCurrency sc:name ?currName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q183'), ('cName', 'Germany'), ('previousCurrency', 'http://www.wikidata.org/entity/Q16068'), ('currName', 'Deutsche Mark')]
[('country', 'http://www.wikidata.org/entity/Q29999'), ('cName', 'Kingdom of the Netherlands'), ('previousCurrency', 'http://www.wikidata.org/entity/Q788472'), ('currName', 'Dutch guilder')]
[('country', 'http://www.wikidata.org/entity/Q33'), ('cName', 'Finland'), ('previousCurrency', 'http://www.wikidata.org/entity/Q203354'), ('currName', 'Finnish markka')]
[('country', 'http://www.wikidata.org/entity/Q27'), ('cName', 'Ireland'), ('previousCurrency', 'http://www.wikidata.org/entity/Q270642'), ('currName', 'Irish pound')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('cName', 'France'), ('previousCurrency', 'http://www.wikidata.org/entity/Q184172'), ('currName', 'French franc')]
[('country', 'http://www.wikidata.org/entity/Q228'), ('cName', 'Andorra'), ('previousCurrency', 'http://www.wikidata.org/entity

In [15]:
#For each country (with * operator) that now uses Euro return which was the currencies used before
queryString = """
SELECT DISTINCT ?country ?cName ?previousCurrency ?currName
WHERE {
   ?country wdt:P31 ?class;
            wdt:P38 wd:Q4916.
   ?class wdt:P279* wd:Q6256.
   ?previousCurrency wdt:P1366 wd:Q4916.
   ?country wdt:P38 ?previousCurrency.
   
   # get the label
   ?country sc:name ?cName.
   ?previousCurrency sc:name ?currName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [17]:
#For each country (with * operator) that now uses Euro return which was the currencies used before with optional previous currency
queryString = """
SELECT DISTINCT ?country ?cName ?previousCurrency ?currName
WHERE {
   ?country wdt:P31 ?class;
            wdt:P38 wd:Q4916.
   ?class wdt:P279* wd:Q6256.
   OPTIONAL{?previousCurrency wdt:P1366 wd:Q4916;
                     wdt:P17 ?country;
                     sc:name ?currName.}            
   
   # get the label
   ?country sc:name ?cName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q142'), ('cName', 'France'), ('previousCurrency', 'http://www.wikidata.org/entity/Q184172'), ('currName', 'French franc')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('cName', 'Germany'), ('previousCurrency', 'http://www.wikidata.org/entity/Q16068'), ('currName', 'Deutsche Mark')]
[('country', 'http://www.wikidata.org/entity/Q191'), ('cName', 'Estonia'), ('previousCurrency', 'http://www.wikidata.org/entity/Q129983'), ('currName', 'Estonian kroon')]
[('country', 'http://www.wikidata.org/entity/Q211'), ('cName', 'Latvia'), ('previousCurrency', 'http://www.wikidata.org/entity/Q133011'), ('currName', 'Latvian lats')]
[('country', 'http://www.wikidata.org/entity/Q214'), ('cName', 'Slovakia'), ('previousCurrency', 'http://www.wikidata.org/entity/Q178874'), ('currName', 'Slovak koruna')]
[('country', 'http://www.wikidata.org/entity/Q215'), ('cName', 'Slovenia'), ('previousCurrency', 'http://www.wikidata.org/entity/Q208133'), ('cur

In [18]:
#For each country that now uses Euro return which were the currencies used before
queryString = """
SELECT DISTINCT ?country ?cName ?previousCurrency ?currName
WHERE {
   ?country wdt:P31/wdt:P279* wd:Q6256;
            wdt:P38 wd:Q4916.
   ?previousCurrency wdt:P1366 wd:Q4916;
                     wdt:P17 ?country.
   
   # get the label
   ?country sc:name ?cName.
   ?previousCurrency sc:name ?currName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q183'), ('cName', 'Germany'), ('previousCurrency', 'http://www.wikidata.org/entity/Q16068'), ('currName', 'Deutsche Mark')]
[('country', 'http://www.wikidata.org/entity/Q29999'), ('cName', 'Kingdom of the Netherlands'), ('previousCurrency', 'http://www.wikidata.org/entity/Q788472'), ('currName', 'Dutch guilder')]
[('country', 'http://www.wikidata.org/entity/Q33'), ('cName', 'Finland'), ('previousCurrency', 'http://www.wikidata.org/entity/Q203354'), ('currName', 'Finnish markka')]
[('country', 'http://www.wikidata.org/entity/Q27'), ('cName', 'Ireland'), ('previousCurrency', 'http://www.wikidata.org/entity/Q270642'), ('currName', 'Irish pound')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('cName', 'France'), ('previousCurrency', 'http://www.wikidata.org/entity/Q184172'), ('currName', 'French franc')]
[('country', 'http://www.wikidata.org/entity/Q228'), ('cName', 'Andorra'), ('previousCurrency', 'http://www.wikidata.org/entity

Final query for this task

In [19]:
# write your final query
queryString = """
SELECT DISTINCT ?country ?cName ?previousCurrency ?currName
WHERE {
   ?country wdt:P31/wdt:P279* wd:Q6256;
            wdt:P38 wd:Q4916.
   ?previousCurrency wdt:P1366 wd:Q4916;
                     wdt:P17 ?country.
   
   # get the label
   ?country sc:name ?cName.
   ?previousCurrency sc:name ?currName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q183'), ('cName', 'Germany'), ('previousCurrency', 'http://www.wikidata.org/entity/Q16068'), ('currName', 'Deutsche Mark')]
[('country', 'http://www.wikidata.org/entity/Q29999'), ('cName', 'Kingdom of the Netherlands'), ('previousCurrency', 'http://www.wikidata.org/entity/Q788472'), ('currName', 'Dutch guilder')]
[('country', 'http://www.wikidata.org/entity/Q33'), ('cName', 'Finland'), ('previousCurrency', 'http://www.wikidata.org/entity/Q203354'), ('currName', 'Finnish markka')]
[('country', 'http://www.wikidata.org/entity/Q27'), ('cName', 'Ireland'), ('previousCurrency', 'http://www.wikidata.org/entity/Q270642'), ('currName', 'Irish pound')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('cName', 'France'), ('previousCurrency', 'http://www.wikidata.org/entity/Q184172'), ('currName', 'French franc')]
[('country', 'http://www.wikidata.org/entity/Q228'), ('cName', 'Andorra'), ('previousCurrency', 'http://www.wikidata.org/entity

## Task 5
Return all types of dollar in the world (US Dollar, Australian Dollar, ecc...)

In [22]:
#return currencies whose name contains [dD]ollar
queryString = """
SELECT ?o ?name
WHERE {
   ?s wdt:P38 ?o.
   
   # get the label
   ?o sc:name ?name
   FILTER REGEX(?name,".*[dD]ollar.*")
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q3712659'), ('name', 'Texas Dollar')]
[('o', 'http://www.wikidata.org/entity/Q209792'), ('name', 'Jamaican dollar')]
[('o', 'http://www.wikidata.org/entity/Q209792'), ('name', 'Jamaican dollar')]
[('o', 'http://www.wikidata.org/entity/Q392858'), ('name', 'Rhodesian dollar')]
[('o', 'http://www.wikidata.org/entity/Q392858'), ('name', 'Rhodesian dollar')]
[('o', 'http://www.wikidata.org/entity/Q1104069'), ('name', 'Canadian dollar')]
[('o', 'http://www.wikidata.org/entity/Q1104069'), ('name', 'Canadian dollar')]
[('o', 'http://www.wikidata.org/entity/Q1104069'), ('name', 'Canadian dollar')]
[('o', 'http://www.wikidata.org/entity/Q1104069'), ('name', 'Canadian dollar')]
[('o', 'http://www.wikidata.org/entity/Q1104069'), ('name', 'Canadian dollar')]
[('o', 'http://www.wikidata.org/entity/Q1104069'), ('name', 'Canadian dollar')]
[('o', 'http://www.wikidata.org/entity/Q1104069'), ('name', 'Canadian dollar')]
[('o', 'http://www.wikidata.org/entit

In [23]:
# add distinct
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   ?s wdt:P38 ?o.
   
   # get the label
   ?o sc:name ?name
   FILTER REGEX(?name,".*[dD]ollar.*")
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q3712659'), ('name', 'Texas Dollar')]
[('o', 'http://www.wikidata.org/entity/Q209792'), ('name', 'Jamaican dollar')]
[('o', 'http://www.wikidata.org/entity/Q392858'), ('name', 'Rhodesian dollar')]
[('o', 'http://www.wikidata.org/entity/Q1104069'), ('name', 'Canadian dollar')]
[('o', 'http://www.wikidata.org/entity/Q259502'), ('name', 'Australian dollar')]
[('o', 'http://www.wikidata.org/entity/Q4597'), ('name', 'Solomon Islands dollar')]
[('o', 'http://www.wikidata.org/entity/Q4917'), ('name', 'United States dollar')]
[('o', 'http://www.wikidata.org/entity/Q26365'), ('name', 'Eastern Caribbean dollar')]
[('o', 'http://www.wikidata.org/entity/Q650834'), ('name', 'Japanese government-issued dollar in Singapore, Malaya, North Borneo, Sarawak and Brunei')]
[('o', 'http://www.wikidata.org/entity/Q2411666'), ('name', 'Newfoundland dollar')]
[('o', 'http://www.wikidata.org/entity/Q242988'), ('name', 'Liberian dollar')]
[('o', 'http://www.wikidata

In [24]:
#return subject with currency Japanese government-issued dollar in Singapore, Malaya, North Borneo, Sarawak and Brunei
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   ?s wdt:P38 wd:Q650834.
   
   # get the label
   ?s sc:name ?name
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q715645'), ('name', 'Japanese occupation of Singapore')]
[('s', 'http://www.wikidata.org/entity/Q713768'), ('name', 'Japanese occupation of Malaya')]
[('s', 'http://www.wikidata.org/entity/Q15243087'), ('name', 'Japanese occupation of British Borneo')]
3


In [27]:
# remove obsolete currencies
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   ?s wdt:P38 ?o.
   
   # get the label
   ?o sc:name ?name.
   FILTER REGEX(?name,".*[dD]ollar.*")
   FILTER(NOT EXISTS {?o wdt:P31 wd:Q28783456})
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q3712659'), ('name', 'Texas Dollar')]
[('o', 'http://www.wikidata.org/entity/Q209792'), ('name', 'Jamaican dollar')]
[('o', 'http://www.wikidata.org/entity/Q1104069'), ('name', 'Canadian dollar')]
[('o', 'http://www.wikidata.org/entity/Q259502'), ('name', 'Australian dollar')]
[('o', 'http://www.wikidata.org/entity/Q4597'), ('name', 'Solomon Islands dollar')]
[('o', 'http://www.wikidata.org/entity/Q4917'), ('name', 'United States dollar')]
[('o', 'http://www.wikidata.org/entity/Q26365'), ('name', 'Eastern Caribbean dollar')]
[('o', 'http://www.wikidata.org/entity/Q650834'), ('name', 'Japanese government-issued dollar in Singapore, Malaya, North Borneo, Sarawak and Brunei')]
[('o', 'http://www.wikidata.org/entity/Q2411666'), ('name', 'Newfoundland dollar')]
[('o', 'http://www.wikidata.org/entity/Q242988'), ('name', 'Liberian dollar')]
[('o', 'http://www.wikidata.org/entity/Q261447'), ('name', 'Straits dollar')]
[('o', 'http://www.wikidata.o

In [16]:
# try with instance of currency
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   ?s wdt:P31 wd:Q8142.
   
   # get the label
   ?o sc:name ?name
   FILTER REGEX(?name,".*[dD]ollar.*")
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q16194032'), ('name', 'John Dollar')]
[('o', 'http://www.wikidata.org/entity/Q17005047'), ('name', 'Elgin, Illinois, Centennial half dollar')]
[('o', 'http://www.wikidata.org/entity/Q17013886'), ('name', 'Million Dollar Minute')]
[('o', 'http://www.wikidata.org/entity/Q17022048'), ('name', 'Million Dollar Arm')]
[('o', 'http://www.wikidata.org/entity/Q17778146'), ('name', 'Dollar, Harviestoun Road, East Lodge')]
[('o', 'http://www.wikidata.org/entity/Q17778294'), ('name', 'Dollar, 1-5 Academy Place')]
[('o', 'http://www.wikidata.org/entity/Q17778380'), ('name', 'Dollar, 1 Devon Road')]
[('o', 'http://www.wikidata.org/entity/Q17778450'), ('name', 'Dollar, 1 Bridge Street, Seberham')]
[('o', 'http://www.wikidata.org/entity/Q17778578'), ('name', 'Dollar, 15 High Street')]
[('o', 'http://www.wikidata.org/entity/Q17778678'), ('name', 'Dollar, Bridge Street, Telephone Call Box')]
[('o', 'http://www.wikidata.org/entity/Q17778773'), ('name', 'Doll

In [36]:
# return what united states dollar is instance of
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q4917 wdt:P31 ?o.
   
   ?o sc:name ?name
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q14083'), ('name', 'dollar')]
[('o', 'http://www.wikidata.org/entity/Q8142'), ('name', 'currency')]
[('o', 'http://www.wikidata.org/entity/Q756202'), ('name', 'reserve currency')]
3


In [14]:
# return all instances of dollar
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
    ?s wdt:P31* wd:Q14083.
   
   ?s sc:name ?name
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q209792'), ('name', 'Jamaican dollar')]
[('s', 'http://www.wikidata.org/entity/Q392858'), ('name', 'Rhodesian dollar')]
[('s', 'http://www.wikidata.org/entity/Q1104069'), ('name', 'Canadian dollar')]
[('s', 'http://www.wikidata.org/entity/Q14083'), ('name', 'dollar')]
[('s', 'http://www.wikidata.org/entity/Q259502'), ('name', 'Australian dollar')]
[('s', 'http://www.wikidata.org/entity/Q4597'), ('name', 'Solomon Islands dollar')]
[('s', 'http://www.wikidata.org/entity/Q4917'), ('name', 'United States dollar')]
[('s', 'http://www.wikidata.org/entity/Q26365'), ('name', 'Eastern Caribbean dollar')]
[('s', 'http://www.wikidata.org/entity/Q4817670'), ('name', 'Coins and postage stamps of Sealand')]
[('s', 'http://www.wikidata.org/entity/Q2411666'), ('name', 'Newfoundland dollar')]
[('s', 'http://www.wikidata.org/entity/Q242988'), ('name', 'Liberian dollar')]
[('s', 'http://www.wikidata.org/entity/Q4592'), ('name', 'Fijian dollar')]
[('s', 'http

In [15]:
# return all instances of dollar with +
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
    ?s wdt:P31+ wd:Q14083.
   
   ?s sc:name ?name
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q209792'), ('name', 'Jamaican dollar')]
[('s', 'http://www.wikidata.org/entity/Q392858'), ('name', 'Rhodesian dollar')]
[('s', 'http://www.wikidata.org/entity/Q1104069'), ('name', 'Canadian dollar')]
[('s', 'http://www.wikidata.org/entity/Q259502'), ('name', 'Australian dollar')]
[('s', 'http://www.wikidata.org/entity/Q4597'), ('name', 'Solomon Islands dollar')]
[('s', 'http://www.wikidata.org/entity/Q4917'), ('name', 'United States dollar')]
[('s', 'http://www.wikidata.org/entity/Q26365'), ('name', 'Eastern Caribbean dollar')]
[('s', 'http://www.wikidata.org/entity/Q4817670'), ('name', 'Coins and postage stamps of Sealand')]
[('s', 'http://www.wikidata.org/entity/Q2411666'), ('name', 'Newfoundland dollar')]
[('s', 'http://www.wikidata.org/entity/Q242988'), ('name', 'Liberian dollar')]
[('s', 'http://www.wikidata.org/entity/Q4592'), ('name', 'Fijian dollar')]
[('s', 'http://www.wikidata.org/entity/Q1472704'), ('name', 'New Zealand dollar')

In [20]:
# return all instances of dollar union countries with "dollar" in object linked by currency property
queryString = """
SELECT DISTINCT ?d ?name WHERE {
    {
    SELECT DISTINCT ?d ?name
    WHERE {
       ?s wdt:P38 ?d.
       ?d sc:name ?name
       FILTER REGEX(?name,".*[dD]ollar.*")
    }
    }
    UNION
    {
    SELECT DISTINCT ?d ?name
    WHERE {
        ?d wdt:P31+ wd:Q14083.

        ?d sc:name ?name
    }
    }
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('d', 'http://www.wikidata.org/entity/Q3712659'), ('name', 'Texas Dollar')]
[('d', 'http://www.wikidata.org/entity/Q209792'), ('name', 'Jamaican dollar')]
[('d', 'http://www.wikidata.org/entity/Q392858'), ('name', 'Rhodesian dollar')]
[('d', 'http://www.wikidata.org/entity/Q1104069'), ('name', 'Canadian dollar')]
[('d', 'http://www.wikidata.org/entity/Q259502'), ('name', 'Australian dollar')]
[('d', 'http://www.wikidata.org/entity/Q4597'), ('name', 'Solomon Islands dollar')]
[('d', 'http://www.wikidata.org/entity/Q4917'), ('name', 'United States dollar')]
[('d', 'http://www.wikidata.org/entity/Q26365'), ('name', 'Eastern Caribbean dollar')]
[('d', 'http://www.wikidata.org/entity/Q650834'), ('name', 'Japanese government-issued dollar in Singapore, Malaya, North Borneo, Sarawak and Brunei')]
[('d', 'http://www.wikidata.org/entity/Q2411666'), ('name', 'Newfoundland dollar')]
[('d', 'http://www.wikidata.org/entity/Q242988'), ('name', 'Liberian dollar')]
[('d', 'http://www.wikidata

Final query for this task

In [18]:
#Return all types of dollar in the world (US Dollar, Australian Dollar, ecc...)
# write your final query
queryString = """
SELECT DISTINCT ?d ?name WHERE {
    {
    SELECT DISTINCT ?d ?name
    WHERE {
       ?s wdt:P38 ?d.
       ?d sc:name ?name
       FILTER REGEX(?name,".*[dD]ollar.*")
    }
    }
    UNION
    {
    SELECT DISTINCT ?d ?name
    WHERE {
        ?d wdt:P31+ wd:Q14083.

        ?d sc:name ?name
    }
    }
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('d', 'http://www.wikidata.org/entity/Q3712659'), ('name', 'Texas Dollar')]
[('d', 'http://www.wikidata.org/entity/Q209792'), ('name', 'Jamaican dollar')]
[('d', 'http://www.wikidata.org/entity/Q392858'), ('name', 'Rhodesian dollar')]
[('d', 'http://www.wikidata.org/entity/Q1104069'), ('name', 'Canadian dollar')]
[('d', 'http://www.wikidata.org/entity/Q259502'), ('name', 'Australian dollar')]
[('d', 'http://www.wikidata.org/entity/Q4597'), ('name', 'Solomon Islands dollar')]
[('d', 'http://www.wikidata.org/entity/Q4917'), ('name', 'United States dollar')]
[('d', 'http://www.wikidata.org/entity/Q26365'), ('name', 'Eastern Caribbean dollar')]
[('d', 'http://www.wikidata.org/entity/Q650834'), ('name', 'Japanese government-issued dollar in Singapore, Malaya, North Borneo, Sarawak and Brunei')]
[('d', 'http://www.wikidata.org/entity/Q2411666'), ('name', 'Newfoundland dollar')]
[('d', 'http://www.wikidata.org/entity/Q242988'), ('name', 'Liberian dollar')]
[('d', 'http://www.wikidata

## Task 6
Return the currencies used in at least 2 different countries (return also the number of countries using each currency).

In [26]:
#first try of the final query
queryString = """
SELECT ?curr ?name (COUNT(?country) AS ?totCountries)
WHERE {
   ?country wdt:P31 wd:Q6256;
            wdt:P38 ?curr.
   
   # get the label
   ?curr sc:name ?name.
}
GROUP BY ?curr ?name 
HAVING (COUNT(?country) >= 2)
ORDER BY DESC (?totCountries)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('curr', 'http://www.wikidata.org/entity/Q4916'), ('name', 'euro'), ('totCountries', '15')]
[('curr', 'http://www.wikidata.org/entity/Q4917'), ('name', 'United States dollar'), ('totCountries', '11')]
[('curr', 'http://www.wikidata.org/entity/Q861690'), ('name', 'West African CFA franc'), ('totCountries', '8')]
[('curr', 'http://www.wikidata.org/entity/Q26365'), ('name', 'Eastern Caribbean dollar'), ('totCountries', '6')]
[('curr', 'http://www.wikidata.org/entity/Q259502'), ('name', 'Australian dollar'), ('totCountries', '4')]
[('curr', 'http://www.wikidata.org/entity/Q847739'), ('name', 'Central African CFA franc'), ('totCountries', '4')]
[('curr', 'http://www.wikidata.org/entity/Q181907'), ('name', 'South African rand'), ('totCountries', '3')]
[('curr', 'http://www.wikidata.org/entity/Q1472704'), ('name', 'New Zealand dollar'), ('totCountries', '3')]
[('curr', 'http://www.wikidata.org/entity/Q25417'), ('name', 'Danish krone'), ('totCountries', '2')]
[('curr', 'http://www.wik

In [19]:
#try with * operator
queryString = """
SELECT ?curr ?name (COUNT(?country) AS ?totCountries)
WHERE {
   ?country wdt:P31 ?class;
            wdt:P38 ?curr.
   ?class wdt:P279* wd:Q6256.
   
   # get the label
   ?curr sc:name ?name.
}
GROUP BY ?curr ?name 
HAVING (COUNT(?country) >= 2)
ORDER BY DESC (?totCountries)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('curr', 'http://www.wikidata.org/entity/Q4916'), ('name', 'euro'), ('totCountries', '102')]
[('curr', 'http://www.wikidata.org/entity/Q4917'), ('name', 'United States dollar'), ('totCountries', '52')]
[('curr', 'http://www.wikidata.org/entity/Q26365'), ('name', 'Eastern Caribbean dollar'), ('totCountries', '46')]
[('curr', 'http://www.wikidata.org/entity/Q861690'), ('name', 'West African CFA franc'), ('totCountries', '38')]
[('curr', 'http://www.wikidata.org/entity/Q259502'), ('name', 'Australian dollar'), ('totCountries', '26')]
[('curr', 'http://www.wikidata.org/entity/Q847739'), ('name', 'Central African CFA franc'), ('totCountries', '24')]
[('curr', 'http://www.wikidata.org/entity/Q25417'), ('name', 'Danish krone'), ('totCountries', '20')]
[('curr', 'http://www.wikidata.org/entity/Q25224'), ('name', 'pound sterling'), ('totCountries', '20')]
[('curr', 'http://www.wikidata.org/entity/Q199462'), ('name', 'Egyptian pound'), ('totCountries', '16')]
[('curr', 'http://www.wikid

In [23]:
#see countries for euro
queryString = """
SELECT ?country ?name 
WHERE {
   ?country wdt:P31 ?class;
            wdt:P38 wd:Q4916.
   ?class wdt:P279* wd:Q6256.
   
   # get the label
   ?country sc:name ?name.
}
GROUP BY ?country ?name 
HAVING (COUNT(?country) >= 2)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q38'), ('name', 'Italy')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany')]
[('country', 'http://www.wikidata.org/entity/Q233'), ('name', 'Malta')]
[('country', 'http://www.wikidata.org/entity/Q41'), ('name', 'Greece')]
[('country', 'http://www.wikidata.org/entity/Q236'), ('name', 'Montenegro')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
[('country', 'http://www.wikidata.org/entity/Q33'), ('name', 'Finland')]
[('country', 'http://www.wikidata.org/entity/Q40'), ('name', 'Austria')]
[('country', 'http://www.wikidata.org/entity/Q229'), ('name', 'Cyprus')]
[('country', 'http://www.wikidata.org/entity/Q29999'), ('name', 'Kingdom of the Netherlands')]
[('country', 'http://www.wikidata.org/entity/Q45'), ('name', 'Portugal')]
[('country', 'http://www.wikidata.org/entity/Q214'), ('name', 'Slovakia')]
[('country', 'http://www.wikidata.org/entity/Q27'), ('name', 'Ireland')]
[('country', 'htt

In [24]:
#see countries for euro (without having clause)
queryString = """
SELECT ?country ?name 
WHERE {
   ?country wdt:P31 ?class;
            wdt:P38 wd:Q4916.
   ?class wdt:P279* wd:Q6256.
   
   # get the label
   ?country sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany')]
[('country', 'http://www.wikidata.org/entity/Q191'), ('name', 'Estonia')]
[('country', 'http://www.wikidata.org/entity/Q211'), ('name', 'Latvia')]
[('country', 'http://www.wikidata.org/entity/Q211'), ('name', 'Latvia')]
[('country', 'http://www.wikidata.org/entity/Q211'), ('name', 'Latvia')]
[('country', 'http://www.wikidata.org/entity/Q214'), ('name', 'Slovakia')]
[('country', 'http://www.wikidata.org/entity/Q214'), ('name', 'Slovakia')]
[('country', 'http://www.wikidata.org/entity/Q214'), ('name', 'Slovakia')]
[('country', 'http://www.wikidata.

In [87]:
#ADD DISTINCT
queryString = """
SELECT DISTINCT ?country ?name 
WHERE {
   ?country wdt:P31 ?class;
            wdt:P38 wd:Q4916.
   ?class wdt:P279* wd:Q6256.
   
   # get the label
   ?country sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany')]
[('country', 'http://www.wikidata.org/entity/Q191'), ('name', 'Estonia')]
[('country', 'http://www.wikidata.org/entity/Q211'), ('name', 'Latvia')]
[('country', 'http://www.wikidata.org/entity/Q214'), ('name', 'Slovakia')]
[('country', 'http://www.wikidata.org/entity/Q215'), ('name', 'Slovenia')]
[('country', 'http://www.wikidata.org/entity/Q228'), ('name', 'Andorra')]
[('country', 'http://www.wikidata.org/entity/Q229'), ('name', 'Cyprus')]
[('country', 'http://www.wikidata.org/entity/Q233'), ('name', 'Malta')]
[('country', 'http://www.wikidata.org/entity/Q235'), ('name', 'Monaco')]
[('country', 'http://www.wikidata.org/entity/Q236'), ('name', 'Montenegro')]
[('country', 'http://www.wikidata.org/entity/Q237'), ('name', 'Vatican City')]
[('country', 'http://www.wikidata.org/entity/Q238'), ('name', 'San Marino')]
[('country', 'http://ww

In [10]:
#ADD COUNT
queryString = """
SELECT DISTINCT ?country ?name (COUNT(?country) AS ?total)
WHERE {
   ?country wdt:P31 ?class;
            wdt:P38 wd:Q4916.
   ?class wdt:P279* wd:Q6256.
   
   # get the label
   ?country sc:name ?name.
}
GROUP BY ?country ?name
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q215'), ('name', 'Slovenia'), ('total', '1')]
[('country', 'http://www.wikidata.org/entity/Q235'), ('name', 'Monaco'), ('total', '1')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('name', 'Italy'), ('total', '2')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany'), ('total', '2')]
[('country', 'http://www.wikidata.org/entity/Q233'), ('name', 'Malta'), ('total', '2')]
[('country', 'http://www.wikidata.org/entity/Q238'), ('name', 'San Marino'), ('total', '1')]
[('country', 'http://www.wikidata.org/entity/Q37'), ('name', 'Lithuania'), ('total', '1')]
[('country', 'http://www.wikidata.org/entity/Q41'), ('name', 'Greece'), ('total', '2')]
[('country', 'http://www.wikidata.org/entity/Q31'), ('name', 'Belgium'), ('total', '1')]
[('country', 'http://www.wikidata.org/entity/Q236'), ('name', 'Montenegro'), ('total', '4')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('name', 'Spain'), ('total', '1')]
[('cou

In [9]:
#Add outer query to count correctly 
queryString = """
SELECT ?currency ?currencyName (COUNT(?country) AS ?totalCountries) WHERE {
    {    
    SELECT DISTINCT ?country ?currency ?currencyName
    WHERE {
       ?country wdt:P31 ?class;
                wdt:P38 ?currency.
       ?class wdt:P279* wd:Q6256.

       # get the label
       ?currency sc:name ?currencyName.
    }
}
}
GROUP BY ?currency ?currencyName
HAVING (COUNT(?country) >= 2)
ORDER BY DESC (?totalCountries)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('currency', 'http://www.wikidata.org/entity/Q4916'), ('currencyName', 'euro'), ('totalCountries', '25')]
[('currency', 'http://www.wikidata.org/entity/Q4917'), ('currencyName', 'United States dollar'), ('totalCountries', '12')]
[('currency', 'http://www.wikidata.org/entity/Q861690'), ('currencyName', 'West African CFA franc'), ('totalCountries', '8')]
[('currency', 'http://www.wikidata.org/entity/Q26365'), ('currencyName', 'Eastern Caribbean dollar'), ('totalCountries', '6')]
[('currency', 'http://www.wikidata.org/entity/Q847739'), ('currencyName', 'Central African CFA franc'), ('totalCountries', '6')]
[('currency', 'http://www.wikidata.org/entity/Q259502'), ('currencyName', 'Australian dollar'), ('totalCountries', '4')]
[('currency', 'http://www.wikidata.org/entity/Q199462'), ('currencyName', 'Egyptian pound'), ('totalCountries', '4')]
[('currency', 'http://www.wikidata.org/entity/Q199471'), ('currencyName', 'Afghan afghani'), ('totalCountries', '3')]
[('currency', 'http://w

In [17]:
#Add distinct on count and not outer query
queryString = """    
SELECT ?currency ?currencyName (COUNT(DISTINCT ?country) AS ?totalCountries)
    WHERE {
       ?country wdt:P31 ?class;
                wdt:P38 ?currency.
       ?class wdt:P279* wd:Q6256.

       # get the label
       ?currency sc:name ?currencyName.
    }

GROUP BY ?currency ?currencyName
HAVING ( COUNT(DISTINCT ?country) >= 2)
ORDER BY DESC (?totalCountries)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('currency', 'http://www.wikidata.org/entity/Q4916'), ('currencyName', 'euro'), ('totalCountries', '25')]
[('currency', 'http://www.wikidata.org/entity/Q4917'), ('currencyName', 'United States dollar'), ('totalCountries', '12')]
[('currency', 'http://www.wikidata.org/entity/Q861690'), ('currencyName', 'West African CFA franc'), ('totalCountries', '8')]
[('currency', 'http://www.wikidata.org/entity/Q26365'), ('currencyName', 'Eastern Caribbean dollar'), ('totalCountries', '6')]
[('currency', 'http://www.wikidata.org/entity/Q847739'), ('currencyName', 'Central African CFA franc'), ('totalCountries', '6')]
[('currency', 'http://www.wikidata.org/entity/Q259502'), ('currencyName', 'Australian dollar'), ('totalCountries', '4')]
[('currency', 'http://www.wikidata.org/entity/Q199462'), ('currencyName', 'Egyptian pound'), ('totalCountries', '4')]
[('currency', 'http://www.wikidata.org/entity/Q181907'), ('currencyName', 'South African rand'), ('totalCountries', '3')]
[('currency', 'http

In [22]:
 #Add distinct group concat to see list of countries
queryString = """    
    SELECT ?currency ?currencyName (COUNT(DISTINCT ?country) AS ?totalCountries) (GROUP_CONCAT( ?countryName ; separator =", " ) AS ?list)
    WHERE {
       ?country wdt:P31 ?class;
                wdt:P38 ?currency.
       ?class wdt:P279* wd:Q6256.

       # get the label
       ?currency sc:name ?currencyName.
       ?country sc:name ?countryName.
    }

GROUP BY ?currency ?currencyName
HAVING ( COUNT(DISTINCT ?country) >= 2)
ORDER BY DESC (?totalCountries)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('currency', 'http://www.wikidata.org/entity/Q4916'), ('currencyName', 'euro'), ('totalCountries', '25'), ('list', 'France, France, France, France, Germany, Germany, Estonia, Latvia, Latvia, Latvia, Slovakia, Slovakia, Slovakia, Slovenia, Andorra, Andorra, Cyprus, Cyprus, Malta, Malta, Monaco, Montenegro, Montenegro, Montenegro, Montenegro, Vatican City, San Marino, Ireland, Ireland, Ireland, Spain, Kingdom of the Netherlands, Kingdom of the Netherlands, Kingdom of the Netherlands, Belgium, Luxembourg, Finland, Finland, Finland, Lithuania, Italy, Italy, Austria, Austria, Austria, Greece, Greece, Portugal, Portugal, Portugal, Netherlands')]
[('currency', 'http://www.wikidata.org/entity/Q4917'), ('currencyName', 'United States dollar'), ('totalCountries', '12'), ('list', 'Northern Mariana Islands, Liberia, Liberia, American Samoa, United States of America, United States of America, East Timor, East Timor, East Timor, Palau, Palau, Palau, Federated States of Micronesia, Federated

In [23]:
 #Add distinct on list (CANDIDATE FINAL QUERY)
queryString = """    
    SELECT ?currency ?currencyName (COUNT(DISTINCT ?country) AS ?totalCountries) (GROUP_CONCAT(DISTINCT ?countryName ; separator =", " ) AS ?list)
    WHERE {
       ?country wdt:P31 ?class;
                wdt:P38 ?currency.
       ?class wdt:P279* wd:Q6256.

       # get the label
       ?currency sc:name ?currencyName.
       ?country sc:name ?countryName.
    }

GROUP BY ?currency ?currencyName
HAVING ( COUNT(DISTINCT ?country) >= 2)
ORDER BY DESC (?totalCountries)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('currency', 'http://www.wikidata.org/entity/Q4916'), ('currencyName', 'euro'), ('totalCountries', '25'), ('list', 'Andorra, Austria, Belgium, Cyprus, Estonia, Finland, France, Germany, Greece, Ireland, Italy, Kingdom of the Netherlands, Latvia, Lithuania, Luxembourg, Malta, Monaco, Montenegro, Netherlands, Portugal, San Marino, Slovakia, Slovenia, Spain, Vatican City')]
[('currency', 'http://www.wikidata.org/entity/Q4917'), ('currencyName', 'United States dollar'), ('totalCountries', '12'), ('list', 'American Samoa, East Timor, Ecuador, El Salvador, Federated States of Micronesia, Liberia, Marshall Islands, Northern Mariana Islands, Palau, Panama, United States of America, Zimbabwe')]
[('currency', 'http://www.wikidata.org/entity/Q861690'), ('currencyName', 'West African CFA franc'), ('totalCountries', '8'), ('list', 'Benin, Burkina Faso, Guinea-Bissau, Ivory Coast, Mali, Niger, Senegal, Togo')]
[('currency', 'http://www.wikidata.org/entity/Q26365'), ('currencyName', 'Eastern

In [25]:
#different version starting from currencies and country property 
queryString = """    
SELECT ?currency ?currencyName (COUNT(DISTINCT ?country) AS ?totalCountries) (GROUP_CONCAT(DISTINCT ?countryName ; separator =", " ) AS ?list)
WHERE {
      ?currency wdt:P31 wd:Q8142.
      ?currency wdt:P17 ?country.
      # get the label
      ?currency sc:name ?currencyName.
      ?country sc:name ?countryName.
}
GROUP BY ?currency ?currencyName
HAVING ( COUNT(DISTINCT ?country) >= 2)
ORDER BY DESC (?totalCountries)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('currency', 'http://www.wikidata.org/entity/Q4916'), ('currencyName', 'euro'), ('totalCountries', '23'), ('list', 'Andorra, Austria, Belgium, Cyprus, Estonia, Finland, France, Germany, Greece, Ireland, Italy, Latvia, Lithuania, Luxembourg, Malta, Monaco, Netherlands, Portugal, San Marino, Slovakia, Slovenia, Spain, Vatican City')]
[('currency', 'http://www.wikidata.org/entity/Q189097'), ('currencyName', 'peseta'), ('totalCountries', '9'), ('list', 'Andorra, Captaincy General of Cuba, Captaincy General of Puerto Rico, Captaincy General of the Philippines, Equatorial Guinea, Ifni, Spain, Spanish Protectorate in Morocco, Western Sahara')]
[('currency', 'http://www.wikidata.org/entity/Q26365'), ('currencyName', 'Eastern Caribbean dollar'), ('totalCountries', '8'), ('list', 'Anguilla, Antigua and Barbuda, Dominica, Grenada, Montserrat, Saint Kitts and Nevis, Saint Lucia, Saint Vincent and the Grenadines')]
[('currency', 'http://www.wikidata.org/entity/Q861690'), ('currencyName', '

In [26]:
#return objects linked by country property to korean yen
queryString = """
SELECT ?o ?name
WHERE {
   wd:Q868106 wdt:P17 ?o.
   ?o sc:name ?name.
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q503585'), ('name', 'Korea under Japanese rule')]
[('o', 'http://www.wikidata.org/entity/Q28233'), ('name', 'Korean Empire')]
2


In [27]:
#return what korean empire is instance of
queryString = """
SELECT ?o ?name
WHERE {
   wd:Q28233 wdt:P31 ?o.
   ?o sc:name ?name.
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q3024240'), ('name', 'historical country')]
1


In [34]:
#different version starting from currencies and country property not including historical countries
queryString = """    
    SELECT ?currency ?currencyName (COUNT(DISTINCT ?country) AS ?totalCountries) (GROUP_CONCAT(DISTINCT ?countryName ; separator =", " ) AS ?list)
    WHERE {
       ?currency wdt:P31 wd:Q8142.
       ?currency wdt:P17 ?country.
       
       # get the label
       ?currency sc:name ?currencyName.
       ?country sc:name ?countryName.
       FILTER(NOT EXISTS {?country wdt:P31 wd:Q3024240})
    }

GROUP BY ?currency ?currencyName
HAVING ( COUNT(DISTINCT ?country) >= 2)
ORDER BY DESC (?totalCountries)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('currency', 'http://www.wikidata.org/entity/Q4916'), ('currencyName', 'euro'), ('totalCountries', '23'), ('list', 'Andorra, Austria, Belgium, Cyprus, Estonia, Finland, France, Germany, Greece, Ireland, Italy, Latvia, Lithuania, Luxembourg, Malta, Monaco, Netherlands, Portugal, San Marino, Slovakia, Slovenia, Spain, Vatican City')]
[('currency', 'http://www.wikidata.org/entity/Q189097'), ('currencyName', 'peseta'), ('totalCountries', '9'), ('list', 'Andorra, Captaincy General of Cuba, Captaincy General of Puerto Rico, Captaincy General of the Philippines, Equatorial Guinea, Ifni, Spain, Spanish Protectorate in Morocco, Western Sahara')]
[('currency', 'http://www.wikidata.org/entity/Q26365'), ('currencyName', 'Eastern Caribbean dollar'), ('totalCountries', '8'), ('list', 'Anguilla, Antigua and Barbuda, Dominica, Grenada, Montserrat, Saint Kitts and Nevis, Saint Lucia, Saint Vincent and the Grenadines')]
[('currency', 'http://www.wikidata.org/entity/Q861690'), ('currencyName', '

In [71]:
#different filter
queryString = """    
    SELECT ?currency ?currencyName (COUNT(DISTINCT ?country) AS ?totalCountries) (GROUP_CONCAT(DISTINCT ?countryName ; separator =", " ) AS ?list)
    WHERE {
       ?currency wdt:P31 wd:Q8142.
       ?currency wdt:P17 ?country.
       ?country wdt:P31+ ?class.
       # get the label
       ?currency sc:name ?currencyName.
       ?country sc:name ?countryName.
       FILTER(?class != wd:Q3024240)
    }

GROUP BY ?currency ?currencyName
HAVING ( COUNT(DISTINCT ?country) >= 2)
ORDER BY DESC (?totalCountries)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('currency', 'http://www.wikidata.org/entity/Q4916'), ('currencyName', 'euro'), ('totalCountries', '23'), ('list', 'Andorra, Austria, Belgium, Cyprus, Estonia, Finland, France, Germany, Greece, Ireland, Italy, Latvia, Lithuania, Luxembourg, Malta, Monaco, Netherlands, Portugal, San Marino, Slovakia, Slovenia, Spain, Vatican City')]
[('currency', 'http://www.wikidata.org/entity/Q189097'), ('currencyName', 'peseta'), ('totalCountries', '9'), ('list', 'Andorra, Captaincy General of Cuba, Captaincy General of Puerto Rico, Captaincy General of the Philippines, Equatorial Guinea, Ifni, Spain, Spanish Protectorate in Morocco, Western Sahara')]
[('currency', 'http://www.wikidata.org/entity/Q26365'), ('currencyName', 'Eastern Caribbean dollar'), ('totalCountries', '8'), ('list', 'Anguilla, Antigua and Barbuda, Dominica, Grenada, Montserrat, Saint Kitts and Nevis, Saint Lucia, Saint Vincent and the Grenadines')]
[('currency', 'http://www.wikidata.org/entity/Q861690'), ('currencyName', '

In [39]:
#return properties for Carlino currency
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q3658911 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 500
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P571'), ('name', 'inception')]
[('p', 'http://www.wikidata.org/prop/direct/P582'), ('name', 'end time')]
[('p', 'http://www.wikidata.org/prop/direct/P935'), ('name', 'Commons gallery')]
9


In [40]:
#return end time for Carlino currency
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q3658911 wdt:P582 ?obj .
}
LIMIT 500
"""

print("Results")
x=run_query(queryString)

Results
[('obj', '1861-01-01T00:00:00Z')]
1


In [72]:
#different filter
queryString = """    
    SELECT ?currency ?currencyName (COUNT(DISTINCT ?country) AS ?totalCountries) (GROUP_CONCAT(DISTINCT ?countryName ; separator =", " ) AS ?list)
    WHERE {
       ?currency wdt:P31 wd:Q8142.
       ?currency wdt:P17 ?country.
       ?country wdt:P31+ ?class.
       # get the label
       ?currency sc:name ?currencyName.
       ?country sc:name ?countryName.
       FILTER(NOT EXISTS {?currency wdt:P582 ?date })
    }

GROUP BY ?currency ?currencyName
HAVING ( COUNT(DISTINCT ?country) >= 2)
ORDER BY DESC (?totalCountries)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('currency', 'http://www.wikidata.org/entity/Q4916'), ('currencyName', 'euro'), ('totalCountries', '23'), ('list', 'Andorra, Austria, Belgium, Cyprus, Estonia, Finland, France, Germany, Greece, Ireland, Italy, Latvia, Lithuania, Luxembourg, Malta, Monaco, Netherlands, Portugal, San Marino, Slovakia, Slovenia, Spain, Vatican City')]
[('currency', 'http://www.wikidata.org/entity/Q26365'), ('currencyName', 'Eastern Caribbean dollar'), ('totalCountries', '8'), ('list', 'Anguilla, Antigua and Barbuda, Dominica, Grenada, Montserrat, Saint Kitts and Nevis, Saint Lucia, Saint Vincent and the Grenadines')]
[('currency', 'http://www.wikidata.org/entity/Q861690'), ('currencyName', 'West African CFA franc'), ('totalCountries', '8'), ('list', 'Benin, Burkina Faso, Guinea-Bissau, Ivory Coast, Mali, Niger, Senegal, Togo')]
[('currency', 'http://www.wikidata.org/entity/Q847739'), ('currencyName', 'Central African CFA franc'), ('totalCountries', '6'), ('list', 'Cameroon, Central African Republ

In [83]:
#adding filtering condition
queryString = """    
    SELECT ?currency ?currencyName (COUNT(DISTINCT ?country) AS ?totalCountries) 
            (GROUP_CONCAT(DISTINCT ?countryName ; separator =", " ) AS ?list)
    WHERE {
       ?currency wdt:P31 wd:Q8142.
       ?currency wdt:P17 ?country.
       ?country wdt:P31* ?class.
       # get the label
       ?currency sc:name ?currencyName.
       ?country sc:name ?countryName.
       FILTER( ?class != wd:Q3024240 && NOT EXISTS {?currency wdt:P582 ?enddate} )
    }

GROUP BY ?currency ?currencyName
HAVING ( COUNT(DISTINCT ?country) >= 2)
ORDER BY DESC (?totalCountries)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('currency', 'http://www.wikidata.org/entity/Q4916'), ('currencyName', 'euro'), ('totalCountries', '23'), ('list', 'Andorra, Austria, Belgium, Cyprus, Estonia, Finland, France, Germany, Greece, Ireland, Italy, Latvia, Lithuania, Luxembourg, Malta, Monaco, Netherlands, Portugal, San Marino, Slovakia, Slovenia, Spain, Vatican City')]
[('currency', 'http://www.wikidata.org/entity/Q26365'), ('currencyName', 'Eastern Caribbean dollar'), ('totalCountries', '8'), ('list', 'Anguilla, Antigua and Barbuda, Dominica, Grenada, Montserrat, Saint Kitts and Nevis, Saint Lucia, Saint Vincent and the Grenadines')]
[('currency', 'http://www.wikidata.org/entity/Q861690'), ('currencyName', 'West African CFA franc'), ('totalCountries', '8'), ('list', 'Benin, Burkina Faso, Guinea-Bissau, Ivory Coast, Mali, Niger, Senegal, Togo')]
[('currency', 'http://www.wikidata.org/entity/Q847739'), ('currencyName', 'Central African CFA franc'), ('totalCountries', '6'), ('list', 'Cameroon, Central African Republ

In [55]:
 #Back to 1st version(CANDIDATE FINAL QUERY)
queryString = """    
    SELECT ?currency ?currencyName (COUNT(DISTINCT ?country) AS ?totalCountries) (GROUP_CONCAT(DISTINCT ?countryName ; separator =", " ) AS ?list)
    WHERE {
       ?country wdt:P31 ?class;
                wdt:P38 ?currency.
       ?class wdt:P279* wd:Q6256.

       # get the label
       ?currency sc:name ?currencyName.
       ?country sc:name ?countryName.
    }

GROUP BY ?currency ?currencyName
HAVING ( COUNT(DISTINCT ?country) >= 2)
ORDER BY DESC (?totalCountries)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('currency', 'http://www.wikidata.org/entity/Q4916'), ('currencyName', 'euro'), ('totalCountries', '25'), ('list', 'Andorra, Austria, Belgium, Cyprus, Estonia, Finland, France, Germany, Greece, Ireland, Italy, Kingdom of the Netherlands, Latvia, Lithuania, Luxembourg, Malta, Monaco, Montenegro, Netherlands, Portugal, San Marino, Slovakia, Slovenia, Spain, Vatican City')]
[('currency', 'http://www.wikidata.org/entity/Q4917'), ('currencyName', 'United States dollar'), ('totalCountries', '12'), ('list', 'American Samoa, East Timor, Ecuador, El Salvador, Federated States of Micronesia, Liberia, Marshall Islands, Northern Mariana Islands, Palau, Panama, United States of America, Zimbabwe')]
[('currency', 'http://www.wikidata.org/entity/Q861690'), ('currencyName', 'West African CFA franc'), ('totalCountries', '8'), ('list', 'Benin, Burkina Faso, Guinea-Bissau, Ivory Coast, Mali, Niger, Senegal, Togo')]
[('currency', 'http://www.wikidata.org/entity/Q26365'), ('currencyName', 'Eastern

In [69]:
#Back to 1st version adding filter for historical countries
queryString = """    
    SELECT ?currency ?currencyName (COUNT(DISTINCT ?country) AS ?totalCountries) (GROUP_CONCAT(DISTINCT ?countryName ; separator =", " ) AS ?list)
    WHERE {
       ?country wdt:P31 ?class;
                wdt:P38 ?currency.
       
       ?class wdt:P279 wd:Q6256.
       
       ?currency wdt:P31 ?currClass.
       FILTER(?class != wd:Q3024240)
       # get the label
       ?currency sc:name ?currencyName.
       ?country sc:name ?countryName.
       
    } 

GROUP BY ?currency ?currencyName
HAVING ( COUNT(DISTINCT ?country) >= 2 )
ORDER BY DESC (?totalCountries)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('currency', 'http://www.wikidata.org/entity/Q4916'), ('currencyName', 'euro'), ('totalCountries', '24'), ('list', 'Andorra, Austria, Belgium, Cyprus, Estonia, Finland, France, Germany, Greece, Ireland, Italy, Kingdom of the Netherlands, Latvia, Lithuania, Luxembourg, Malta, Monaco, Montenegro, Portugal, San Marino, Slovakia, Slovenia, Spain, Vatican City')]
[('currency', 'http://www.wikidata.org/entity/Q4917'), ('currencyName', 'United States dollar'), ('totalCountries', '10'), ('list', 'East Timor, Ecuador, El Salvador, Federated States of Micronesia, Liberia, Marshall Islands, Palau, Panama, United States of America, Zimbabwe')]
[('currency', 'http://www.wikidata.org/entity/Q861690'), ('currencyName', 'West African CFA franc'), ('totalCountries', '8'), ('list', 'Benin, Burkina Faso, Guinea-Bissau, Ivory Coast, Mali, Niger, Senegal, Togo')]
[('currency', 'http://www.wikidata.org/entity/Q26365'), ('currencyName', 'Eastern Caribbean dollar'), ('totalCountries', '6'), ('list', 

In [66]:
#Back to 1st version adding filter (with not exists) for historical countries
queryString = """    
    SELECT ?currency ?currencyName (COUNT(DISTINCT ?country) AS ?totalCountries) (GROUP_CONCAT(DISTINCT ?countryName ; separator =", " ) AS ?list)
    WHERE {
       ?country wdt:P31 ?class;
                wdt:P38 ?currency.
       
       ?class wdt:P279 wd:Q6256.
       
       ?currency wdt:P31 ?currClass.
       FILTER(NOT EXISTS {?country wdt:P31 wd:Q3024240})
       # get the label
       ?currency sc:name ?currencyName.
       ?country sc:name ?countryName.
       
    } 

GROUP BY ?currency ?currencyName
HAVING ( COUNT(DISTINCT ?country) >= 2 )
ORDER BY DESC (?totalCountries)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('currency', 'http://www.wikidata.org/entity/Q4916'), ('currencyName', 'euro'), ('totalCountries', '24'), ('list', 'Andorra, Austria, Belgium, Cyprus, Estonia, Finland, France, Germany, Greece, Ireland, Italy, Kingdom of the Netherlands, Latvia, Lithuania, Luxembourg, Malta, Monaco, Montenegro, Portugal, San Marino, Slovakia, Slovenia, Spain, Vatican City')]
[('currency', 'http://www.wikidata.org/entity/Q4917'), ('currencyName', 'United States dollar'), ('totalCountries', '10'), ('list', 'East Timor, Ecuador, El Salvador, Federated States of Micronesia, Liberia, Marshall Islands, Palau, Panama, United States of America, Zimbabwe')]
[('currency', 'http://www.wikidata.org/entity/Q861690'), ('currencyName', 'West African CFA franc'), ('totalCountries', '8'), ('list', 'Benin, Burkina Faso, Guinea-Bissau, Ivory Coast, Mali, Niger, Senegal, Togo')]
[('currency', 'http://www.wikidata.org/entity/Q26365'), ('currencyName', 'Eastern Caribbean dollar'), ('totalCountries', '6'), ('list', 

In [59]:
# return countries which use Egyptian pound
queryString = """
SELECT DISTINCT ?country?name
WHERE {
   ?country wdt:P31 ?class;
            wdt:P38 wd:Q199462.
   ?class wdt:P279 wd:Q6256.
   ?country sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q127861'), ('name', 'Khedivate of Egypt')]
[('country', 'http://www.wikidata.org/entity/Q370173'), ('name', 'Sultanate of Egypt')]
[('country', 'http://www.wikidata.org/entity/Q124943'), ('name', 'Kingdom of Egypt')]
[('country', 'http://www.wikidata.org/entity/Q79'), ('name', 'Egypt')]
4


In [61]:
#return what Kingdom of Egypt is instance of
queryString = """
SELECT ?o ?name
WHERE {
   wd:Q124943 wdt:P31 ?o.
   ?o sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q1323642'), ('name', 'transcontinental country')]
[('o', 'http://www.wikidata.org/entity/Q3024240'), ('name', 'historical country')]
[('o', 'http://www.wikidata.org/entity/Q1151405'), ('name', 'client state')]
3


In [70]:
#return what Sultanate of Egypt is instance of
queryString = """
SELECT ?o ?name
WHERE {
   wd:Q370173 wdt:P31 ?o.
   ?o sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q1323642'), ('name', 'transcontinental country')]
[('o', 'http://www.wikidata.org/entity/Q21479969'), ('name', 'British protectorate')]
[('o', 'http://www.wikidata.org/entity/Q12759805'), ('name', 'sultanate')]
3


In [71]:
#return property and obj for Sultanate of Egypt
queryString = """
SELECT DISTINCT ?p ?pname ?obj ?name 
WHERE {
   wd:Q370173 ?p ?obj .
   # get the label
   ?obj sc:name ?name.
   ?p sc:name ?pname.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P122'), ('pname', 'basic form of government'), ('obj', 'http://www.wikidata.org/entity/Q41614'), ('name', 'constitutional monarchy')]
[('p', 'http://www.wikidata.org/prop/direct/P1313'), ('pname', 'office held by head of government'), ('obj', 'http://www.wikidata.org/entity/Q4115925'), ('name', 'Sultan of Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('pname', 'replaces'), ('obj', 'http://www.wikidata.org/entity/Q127861'), ('name', 'Khedivate of Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P1366'), ('pname', 'replaced by'), ('obj', 'http://www.wikidata.org/entity/Q124943'), ('name', 'Kingdom of Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P140'), ('pname', 'religion'), ('obj', 'http://www.wikidata.org/entity/Q483654'), ('name', 'Sunni Islam')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pname', 'follows'), ('obj', 'http://www.wikidata.org/entity/Q127861'), ('name', 'Khedivate of Egypt')]
[('p'

In [76]:
# return all properties for Roman currency
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q952064 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('name', 'facet of')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('name', 'BabelNet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de France ID')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P3219'), ('name', 'Encyclopædia Universalis ID')]
[('p', 'http://www.wikidata.org/prop/direct/P3417'), ('name', 'Quora topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P4212'), ('name', 'PACTOLS thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/d

In [77]:
# return what Roman currency is instance of
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q952064 wdt:P31 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q17524420'), ('name', 'aspect of history')]
1


In [8]:
#return property and obj for Roman currency
queryString = """
SELECT DISTINCT ?p ?pname ?obj ?name 
WHERE {
   wd:Q952064 ?p ?obj .
   # get the label
   ?obj sc:name ?name.
   ?p sc:name ?pname.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pname', 'facet of'), ('obj', 'http://www.wikidata.org/entity/Q1747689'), ('name', 'Ancient Rome')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pname', 'facet of'), ('obj', 'http://www.wikidata.org/entity/Q8142'), ('name', 'currency')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pname', 'described by source'), ('obj', 'http://www.wikidata.org/entity/Q602358'), ('name', 'Brockhaus and Efron Encyclopedic Dictionary')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pname', 'described by source'), ('obj', 'http://www.wikidata.org/entity/Q1138524'), ('name', 'Paulys Realenzyklopädie der klassischen Altertumswissenschaft')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pname', 'described by source'), ('obj', 'http://www.wikidata.org/entity/Q30059240'), ('name', "Real'nyj slovar' klassicheskih drevnostej po Ljubkeru")]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of')

In [87]:
# return what is linked by currency property to Roman currency 
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   ?s wdt:P38 wd:Q952064 .
   # get the label
   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q2277'), ('name', 'Roman Empire')]
[('s', 'http://www.wikidata.org/entity/Q42834'), ('name', 'Western Roman Empire (395-476 AD)')]
[('s', 'http://www.wikidata.org/entity/Q17167'), ('name', 'Roman Republic')]
[('s', 'http://www.wikidata.org/entity/Q1247508'), ('name', 'Domain of Soissons')]
4


In [91]:
#return what roman empire is instance of
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q2277 wdt:P31* ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q11514315'), ('name', 'historical period')]
[('obj', 'http://www.wikidata.org/entity/Q3024240'), ('name', 'historical country')]
[('obj', 'http://www.wikidata.org/entity/Q48349'), ('name', 'empire')]
3


In [3]:
#return property and obj for historical country
queryString = """
SELECT DISTINCT ?p ?pname ?obj ?name 
WHERE {
   wd:Q3024240 ?p ?obj .
   # get the label
   ?obj sc:name ?name.
   ?p sc:name ?pname.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pname', "topic's main template"), ('obj', 'http://www.wikidata.org/entity/Q6036853'), ('name', 'Template:Infobox former country')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pname', 'Wikidata property'), ('obj', 'http://www.wikidata.org/entity/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pname', 'different from'), ('obj', 'http://www.wikidata.org/entity/Q28864179'), ('name', 'proposed country')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P1366'), ('name', 'replaced by')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P36'), ('name', 'capital')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P576'), ('name', 'dissolved, a

In [45]:
# return nodes which properties link Germany and countries * operator
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q183 ?p ?o.
   ?o wdt:P279* wd:Q6256.
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
1


In [78]:
#Back to 1st version adding filter (with not exists) for historical countries and not countries replaced by something
queryString = """    
    SELECT ?currency ?currencyName (COUNT(DISTINCT ?country) AS ?totalCountries) (GROUP_CONCAT(DISTINCT ?countryName ; separator =", " ) AS ?list)
    WHERE {
       ?country wdt:P31 ?class;
                wdt:P38 ?currency.
       
       ?class wdt:P279 wd:Q6256.
       
       ?currency wdt:P31 ?currClass.
       FILTER(NOT EXISTS {?country wdt:P31 wd:Q3024240.} && NOT EXISTS {?country wdt:P1366 ?o.})
                                
       # get the label
       ?currency sc:name ?currencyName.
       ?country sc:name ?countryName.
       
    } 

GROUP BY ?currency ?currencyName
HAVING ( COUNT(DISTINCT ?country) >= 2 )
ORDER BY DESC (?totalCountries)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('currency', 'http://www.wikidata.org/entity/Q4916'), ('currencyName', 'euro'), ('totalCountries', '24'), ('list', 'Andorra, Austria, Belgium, Cyprus, Estonia, Finland, France, Germany, Greece, Ireland, Italy, Kingdom of the Netherlands, Latvia, Lithuania, Luxembourg, Malta, Monaco, Montenegro, Portugal, San Marino, Slovakia, Slovenia, Spain, Vatican City')]
[('currency', 'http://www.wikidata.org/entity/Q4917'), ('currencyName', 'United States dollar'), ('totalCountries', '10'), ('list', 'East Timor, Ecuador, El Salvador, Federated States of Micronesia, Liberia, Marshall Islands, Palau, Panama, United States of America, Zimbabwe')]
[('currency', 'http://www.wikidata.org/entity/Q861690'), ('currencyName', 'West African CFA franc'), ('totalCountries', '8'), ('list', 'Benin, Burkina Faso, Guinea-Bissau, Ivory Coast, Mali, Niger, Senegal, Togo')]
[('currency', 'http://www.wikidata.org/entity/Q26365'), ('currencyName', 'Eastern Caribbean dollar'), ('totalCountries', '6'), ('list', 

In [76]:
# return countries which use Danish krone
queryString = """
SELECT DISTINCT ?country?name
WHERE {
   ?country wdt:P31 ?class;
            wdt:P38 wd:Q25417.
   ?class wdt:P279 wd:Q6256.
   ?country sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q223'), ('name', 'Greenland')]
[('country', 'http://www.wikidata.org/entity/Q756617'), ('name', 'Danish Realm')]
[('country', 'http://www.wikidata.org/entity/Q35'), ('name', 'Denmark')]
3


In [77]:
# return what Danish realm is instance of
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q756617 wdt:P31 ?o.
   ?o sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q179164'), ('name', 'unitary state')]
[('o', 'http://www.wikidata.org/entity/Q20181813'), ('name', 'colonial power')]
[('o', 'http://www.wikidata.org/entity/Q3624078'), ('name', 'sovereign state')]
[('o', 'http://www.wikidata.org/entity/Q41614'), ('name', 'constitutional monarchy')]
[('o', 'http://www.wikidata.org/entity/Q6256'), ('name', 'country')]
[('o', 'http://www.wikidata.org/entity/Q417175'), ('name', 'kingdom')]
6


In [104]:
#1st version adding filter (with not exists) for historical countries and not countries replaced by something
#and not instance of kingdom
queryString = """    
    SELECT ?currency ?currencyName (COUNT(DISTINCT ?country) AS ?totalCountries) (GROUP_CONCAT(DISTINCT ?countryName ; separator =", " ) AS ?list)
    WHERE {
       ?country wdt:P31 ?class;
                wdt:P38 ?currency.
       
       ?class wdt:P279 wd:Q6256.
       
       ?currency wdt:P31 wd:Q8142.
       FILTER(NOT EXISTS {?country wdt:P31 wd:Q3024240.} && NOT EXISTS {?country wdt:P1366 ?o.}
                && NOT EXISTS {?country wdt:P31 wd:Q417175.})
                                
       # get the label
       ?currency sc:name ?currencyName.
       ?country sc:name ?countryName.
       
    } 

GROUP BY ?currency ?currencyName
HAVING ( COUNT(DISTINCT ?country) >= 2 )
ORDER BY DESC (?totalCountries)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('currency', 'http://www.wikidata.org/entity/Q4916'), ('currencyName', 'euro'), ('totalCountries', '23'), ('list', 'Andorra, Austria, Belgium, Cyprus, Estonia, Finland, France, Germany, Greece, Ireland, Italy, Latvia, Lithuania, Luxembourg, Malta, Monaco, Montenegro, Portugal, San Marino, Slovakia, Slovenia, Spain, Vatican City')]
[('currency', 'http://www.wikidata.org/entity/Q4917'), ('currencyName', 'United States dollar'), ('totalCountries', '10'), ('list', 'East Timor, Ecuador, El Salvador, Federated States of Micronesia, Liberia, Marshall Islands, Palau, Panama, United States of America, Zimbabwe')]
[('currency', 'http://www.wikidata.org/entity/Q861690'), ('currencyName', 'West African CFA franc'), ('totalCountries', '8'), ('list', 'Benin, Burkina Faso, Guinea-Bissau, Ivory Coast, Mali, Niger, Senegal, Togo')]
[('currency', 'http://www.wikidata.org/entity/Q26365'), ('currencyName', 'Eastern Caribbean dollar'), ('totalCountries', '6'), ('list', 'Antigua and Barbuda, Domini

In [82]:
# return countries which use Russian ruble
queryString = """
SELECT DISTINCT ?country?name
WHERE {
   ?country wdt:P31 ?class;
            wdt:P38 wd:Q41044.
   ?class wdt:P279 wd:Q6256.
   ?country sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q159'), ('name', 'Russia')]
[('country', 'http://www.wikidata.org/entity/Q23427'), ('name', 'South Ossetia')]
2


In [83]:
#return property and obj for South Ossetia
queryString = """
SELECT DISTINCT ?p ?pname ?obj ?name 
WHERE {
   wd:Q23427 ?p ?obj .
   # get the label
   ?obj sc:name ?name.
   ?p sc:name ?pname.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('pname', "topic's main Wikimedia portal"), ('obj', 'http://www.wikidata.org/entity/Q11148074'), ('name', 'Portal:South Ossetia')]
[('p', 'http://www.wikidata.org/prop/direct/P122'), ('pname', 'basic form of government'), ('obj', 'http://www.wikidata.org/entity/Q49892'), ('name', 'presidential régime')]
[('p', 'http://www.wikidata.org/prop/direct/P1313'), ('pname', 'office held by head of government'), ('obj', 'http://www.wikidata.org/entity/Q2332585'), ('name', 'Prime Minister of South Ossetia')]
[('p', 'http://www.wikidata.org/prop/direct/P1336'), ('pname', 'territory claimed by'), ('obj', 'http://www.wikidata.org/entity/Q230'), ('name', 'Georgia')]
[('p', 'http://www.wikidata.org/prop/direct/P1465'), ('pname', 'category for people who died here'), ('obj', 'http://www.wikidata.org/entity/Q9828708'), ('name', 'Category:Deaths in South Ossetia')]
[('p', 'http://www.wikidata.org/prop/direct/P150'), ('pname', 'contains administ

In [105]:
#1st version adding filter (with not exists) for historical countries and not countries replaced by something
#and not instance of kingdom and moved subclass of country match on filter
queryString = """    
    SELECT ?currency ?currencyName (COUNT(DISTINCT ?country) AS ?totalCountries) (GROUP_CONCAT(DISTINCT ?countryName ; separator =", " ) AS ?list)
    WHERE {
       ?country wdt:P31 ?class;
                wdt:P38 ?currency.
       
       ?currency wdt:P31 wd:Q8142.
       FILTER(NOT EXISTS {?country wdt:P31 wd:Q3024240.} && NOT EXISTS {?country wdt:P1366 ?o.}
                && NOT EXISTS {?country wdt:P31 wd:Q417175.} && EXISTS{?class wdt:P279* wd:Q6256.})
                                
       # get the label
       ?currency sc:name ?currencyName.
       ?country sc:name ?countryName.
       
    } 

GROUP BY ?currency ?currencyName
HAVING ( COUNT(DISTINCT ?country) >= 2 )
ORDER BY DESC (?totalCountries)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('currency', 'http://www.wikidata.org/entity/Q4916'), ('currencyName', 'euro'), ('totalCountries', '24'), ('list', 'Andorra, Austria, Belgium, Cyprus, Estonia, Finland, France, Germany, Greece, Ireland, Italy, Latvia, Lithuania, Luxembourg, Malta, Monaco, Montenegro, Netherlands, Portugal, San Marino, Slovakia, Slovenia, Spain, Vatican City')]
[('currency', 'http://www.wikidata.org/entity/Q4917'), ('currencyName', 'United States dollar'), ('totalCountries', '12'), ('list', 'American Samoa, East Timor, Ecuador, El Salvador, Federated States of Micronesia, Liberia, Marshall Islands, Northern Mariana Islands, Palau, Panama, United States of America, Zimbabwe')]
[('currency', 'http://www.wikidata.org/entity/Q861690'), ('currencyName', 'West African CFA franc'), ('totalCountries', '8'), ('list', 'Benin, Burkina Faso, Guinea-Bissau, Ivory Coast, Mali, Niger, Senegal, Togo')]
[('currency', 'http://www.wikidata.org/entity/Q26365'), ('currencyName', 'Eastern Caribbean dollar'), ('total

In [19]:
#Return the currencies used in at least 2 different countries (return also the number of countries using each currency).
# write your final query
queryString = """    
SELECT ?currency ?currencyName (COUNT(DISTINCT ?country) AS ?totalCountries)
    WHERE {
       ?country wdt:P31 ?class;
                wdt:P38 ?currency.
       
       ?currency wdt:P31 wd:Q8142.
       FILTER(NOT EXISTS {?country wdt:P31 wd:Q3024240.} && NOT EXISTS {?country wdt:P1366 ?o.}
                && NOT EXISTS {?country wdt:P31 wd:Q417175.} && EXISTS{?class wdt:P279* wd:Q6256.})
                                
       # get the label
       ?currency sc:name ?currencyName.
       ?country sc:name ?countryName.
    } 

GROUP BY ?currency ?currencyName
HAVING ( COUNT(DISTINCT ?country) >= 2 )
ORDER BY DESC (?totalCountries)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('currency', 'http://www.wikidata.org/entity/Q4916'), ('currencyName', 'euro'), ('totalCountries', '24')]
[('currency', 'http://www.wikidata.org/entity/Q4917'), ('currencyName', 'United States dollar'), ('totalCountries', '12')]
[('currency', 'http://www.wikidata.org/entity/Q861690'), ('currencyName', 'West African CFA franc'), ('totalCountries', '8')]
[('currency', 'http://www.wikidata.org/entity/Q26365'), ('currencyName', 'Eastern Caribbean dollar'), ('totalCountries', '6')]
[('currency', 'http://www.wikidata.org/entity/Q847739'), ('currencyName', 'Central African CFA franc'), ('totalCountries', '6')]
[('currency', 'http://www.wikidata.org/entity/Q259502'), ('currencyName', 'Australian dollar'), ('totalCountries', '4')]
[('currency', 'http://www.wikidata.org/entity/Q1472704'), ('currencyName', 'New Zealand dollar'), ('totalCountries', '3')]
[('currency', 'http://www.wikidata.org/entity/Q130498'), ('currencyName', 'Armenian dram'), ('totalCountries', '2')]
[('currency', 'http

Final query for this task

In [13]:
#Return the currencies used in at least 2 different countries (return also the number of countries using each currency).
# write your final query
queryString = """    
SELECT ?currency ?currencyName (COUNT(DISTINCT ?country) AS ?totalCountries)
    WHERE {
       ?country wdt:P31/wdt:P279* wd:Q6256;
                wdt:P38 ?currency.
       ?currency wdt:P31 wd:Q8142.
       FILTER(NOT EXISTS {?country wdt:P31 wd:Q3024240.} && NOT EXISTS {?country wdt:P1366 ?o.}
                && NOT EXISTS {?country wdt:P31 wd:Q417175.})
                                
       # get the label
       ?currency sc:name ?currencyName.
       ?country sc:name ?countryName.
    } 

GROUP BY ?currency ?currencyName
HAVING ( COUNT(DISTINCT ?country) >= 2 )
ORDER BY DESC (?totalCountries)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('currency', 'http://www.wikidata.org/entity/Q4916'), ('currencyName', 'euro'), ('totalCountries', '24')]
[('currency', 'http://www.wikidata.org/entity/Q4917'), ('currencyName', 'United States dollar'), ('totalCountries', '12')]
[('currency', 'http://www.wikidata.org/entity/Q861690'), ('currencyName', 'West African CFA franc'), ('totalCountries', '8')]
[('currency', 'http://www.wikidata.org/entity/Q26365'), ('currencyName', 'Eastern Caribbean dollar'), ('totalCountries', '6')]
[('currency', 'http://www.wikidata.org/entity/Q847739'), ('currencyName', 'Central African CFA franc'), ('totalCountries', '6')]
[('currency', 'http://www.wikidata.org/entity/Q259502'), ('currencyName', 'Australian dollar'), ('totalCountries', '4')]
[('currency', 'http://www.wikidata.org/entity/Q1472704'), ('currencyName', 'New Zealand dollar'), ('totalCountries', '3')]
[('currency', 'http://www.wikidata.org/entity/Q130498'), ('currencyName', 'Armenian dram'), ('totalCountries', '2')]
[('currency', 'http

## Task 7
Consider the currencies obtained in task 6, for each currency return the number of people using that currency (hint: consider the maximum popolation available for each country).

In [8]:
# return all properties for Italy
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q38 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1081'), ('name', 'Human Development Index')]
[('p', 'http://www.wikidata.org/prop/direct/P1082'), ('name', 'population')]
[('p', 'http://www.wikidata.org/prop/direct/P1120'), ('name', 'number of deaths')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1198'), ('name', 'unemployment rate')]
[('p', 'http://www.wikidata.org/prop/direct/P122'), ('name', 'basic form of government')]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalan

In [12]:
# return what Italy is instance of
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   wd:Q38 wdt:P31 ?obj .
   
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q3624078'), ('name', 'sovereign state')]
[('obj', 'http://www.wikidata.org/entity/Q6256'), ('name', 'country')]
2


In [9]:
# return Italy's population
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   wd:Q38 wdt:P1082 ?obj .
   
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [10]:
# return Italy's population
queryString = """
SELECT DISTINCT ?obj 
WHERE {
   wd:Q38 wdt:P1082 ?obj .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', '60317000')]
1


In [32]:
# return all properties for Euro
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q4916 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1001'), ('name', 'applies to jurisdiction')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1282'), ('name', 'OpenStreetMap tag or key')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1482'), ('name', 'Stack Exchange tag')]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1813'), ('name', 'short name')]
[('p', 

In [27]:
#final query try
queryString = """
SELECT ?curr ?name (SUM(?people) AS ?totalPopulation) WHERE {
    ?curr wdt:P17 ?country.
    ?country wdt:P31 wd:Q6256.
    ?country wdt:P1082 ?people.
    {
    SELECT ?curr ?name (COUNT(?country) AS ?totCountries)
    WHERE {
       ?country wdt:P31 wd:Q6256;
                wdt:P38 ?curr.

       # get the label
       ?curr sc:name ?name.
    }
    GROUP BY ?curr ?name 
    HAVING (COUNT(?country) >= 2)
    }
}
GROUP BY ?curr ?name 
ORDER BY DESC (?totalPopulation)
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('curr', 'http://www.wikidata.org/entity/Q4917'), ('name', 'United States dollar'), ('totalPopulation', '331449281')]
[('curr', 'http://www.wikidata.org/entity/Q4916'), ('name', 'euro'), ('totalPopulation', '193833944')]
[('curr', 'http://www.wikidata.org/entity/Q861690'), ('name', 'West African CFA franc'), ('totalPopulation', '121487314')]
[('curr', 'http://www.wikidata.org/entity/Q188608'), ('name', 'Iranian rial'), ('totalPopulation', '79966230')]
[('curr', 'http://www.wikidata.org/entity/Q25224'), ('name', 'pound sterling'), ('totalPopulation', '66022273')]
[('curr', 'http://www.wikidata.org/entity/Q181907'), ('name', 'South African rand'), ('totalPopulation', '59622350')]
[('curr', 'http://www.wikidata.org/entity/Q847739'), ('name', 'Central African CFA franc'), ('totalPopulation', '49451308')]
[('curr', 'http://www.wikidata.org/entity/Q259502'), ('name', 'Australian dollar'), ('totalPopulation', '24511800')]
[('curr', 'http://www.wikidata.org/entity/Q25344'), ('name', '

In [2]:
# return object linked by country property from United States dollar
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   wd:Q4917 wdt:P17 ?obj .
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q30'), ('name', 'United States of America')]
1


In [21]:
# return countries using United States dollar
queryString = """
SELECT DISTINCT ?country ?name
WHERE {
   ?country wdt:P31/wdt:P279* wd:Q6256;
            wdt:P38 wd:Q4917.
   ?country sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q16644'), ('name', 'Northern Mariana Islands')]
[('country', 'http://www.wikidata.org/entity/Q1014'), ('name', 'Liberia')]
[('country', 'http://www.wikidata.org/entity/Q16641'), ('name', 'American Samoa')]
[('country', 'http://www.wikidata.org/entity/Q30'), ('name', 'United States of America')]
[('country', 'http://www.wikidata.org/entity/Q574'), ('name', 'East Timor')]
[('country', 'http://www.wikidata.org/entity/Q695'), ('name', 'Palau')]
[('country', 'http://www.wikidata.org/entity/Q702'), ('name', 'Federated States of Micronesia')]
[('country', 'http://www.wikidata.org/entity/Q709'), ('name', 'Marshall Islands')]
[('country', 'http://www.wikidata.org/entity/Q736'), ('name', 'Ecuador')]
[('country', 'http://www.wikidata.org/entity/Q792'), ('name', 'El Salvador')]
[('country', 'http://www.wikidata.org/entity/Q804'), ('name', 'Panama')]
[('country', 'http://www.wikidata.org/entity/Q954'), ('name', 'Zimbabwe')]
12


In [45]:
# return United State's population
queryString = """
SELECT DISTINCT ?obj 
WHERE {
   wd:Q30 wdt:P1082 ?obj .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', '331449281')]
1


In [28]:
#final query try (wrong code P37)
queryString = """
SELECT ?curr ?name (SUM(?people) AS ?totalPopulation) WHERE {
    ?country wdt:P37 ?curr.
    ?country wdt:P31 wd:Q6256.
    ?country wdt:P1082 ?people.
    {
    SELECT ?curr ?name (COUNT(?country) AS ?totCountries)
    WHERE {
       ?country wdt:P31 wd:Q6256;
                wdt:P38 ?curr.

       # get the label
       ?curr sc:name ?name.
    }
    GROUP BY ?curr ?name 
    HAVING (COUNT(?country) >= 2)
    }
}
GROUP BY ?curr ?name 
ORDER BY DESC (?totalPopulation)
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [31]:
#final query try 
queryString = """
SELECT ?curr ?name (SUM(?people) AS ?totalPeople) WHERE {
    ?country wdt:P38 ?curr.
    ?country wdt:P31 wd:Q6256.
    ?country wdt:P1082 ?people.
    {
    SELECT ?curr ?name (COUNT(?country) AS ?totCountries)
    WHERE {
       ?country wdt:P31 wd:Q6256;
                wdt:P38 ?curr.

       # get the label
       ?curr sc:name ?name.
    }
    GROUP BY ?curr ?name 
    HAVING (COUNT(?country) >= 2)
    }
}
GROUP BY ?curr ?name 
ORDER BY DESC (?totalPeople)
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('curr', 'http://www.wikidata.org/entity/Q4917'), ('name', 'United States dollar'), ('totalPeople', '377344239')]
[('curr', 'http://www.wikidata.org/entity/Q4916'), ('name', 'euro'), ('totalPeople', '211557018')]
[('curr', 'http://www.wikidata.org/entity/Q861690'), ('name', 'West African CFA franc'), ('totalPeople', '121487314')]
[('curr', 'http://www.wikidata.org/entity/Q188608'), ('name', 'Iranian rial'), ('totalPeople', '79966230')]
[('curr', 'http://www.wikidata.org/entity/Q25224'), ('name', 'pound sterling'), ('totalPeople', '69135273')]
[('curr', 'http://www.wikidata.org/entity/Q181907'), ('name', 'South African rand'), ('totalPeople', '64389483')]
[('curr', 'http://www.wikidata.org/entity/Q847739'), ('name', 'Central African CFA franc'), ('totalPeople', '49451308')]
[('curr', 'http://www.wikidata.org/entity/Q259502'), ('name', 'Australian dollar'), ('totalPeople', '24656582')]
[('curr', 'http://www.wikidata.org/entity/Q25417'), ('name', 'Danish krone'), ('totalPeople', 

In [49]:
#final query try on updated task 6
queryString = """
SELECT ?currency ?currencyName (SUM(?people) AS ?totalPeople) WHERE {
    ?currency ^wdt:P38 ?country.
    ?country wdt:P31 ?class.
    ?class wdt:P279* wd:Q6256.
    ?country wdt:P1082 ?people.
    {
            SELECT DISTINCT ?currency ?currencyName
            WHERE {
               ?country wdt:P31 ?class;
                        wdt:P38 ?currency.
               ?class wdt:P279* wd:Q6256.

               # get the label
               ?currency sc:name ?currencyName.
            }
    GROUP BY ?currency ?currencyName
    HAVING (COUNT( ?country) >= 2)
    }
}
GROUP BY ?currency ?currencyName 
ORDER BY DESC (?totalPeople)
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('currency', 'http://www.wikidata.org/entity/Q25224'), ('currencyName', 'pound sterling'), ('totalPeople', '6106448730')]
[('currency', 'http://www.wikidata.org/entity/Q39099'), ('currencyName', 'renminbi'), ('totalPeople', '5773989512')]
[('currency', 'http://www.wikidata.org/entity/Q80524'), ('currencyName', 'Indian rupee'), ('totalPeople', '5304372988')]
[('currency', 'http://www.wikidata.org/entity/Q41588'), ('currencyName', 'rupiah'), ('totalPeople', '2165004544')]
[('currency', 'http://www.wikidata.org/entity/Q4916'), ('currencyName', 'euro'), ('totalPeople', '1652751098')]
[('currency', 'http://www.wikidata.org/entity/Q4917'), ('currencyName', 'United States dollar'), ('totalPeople', '1553365694')]
[('currency', 'http://www.wikidata.org/entity/Q199462'), ('currencyName', 'Egyptian pound'), ('totalPeople', '973490270')]
[('currency', 'http://www.wikidata.org/entity/Q8146'), ('currencyName', 'Japanese yen'), ('totalPeople', '954147390')]
[('currency', 'http://www.wikidata

In [35]:
#return which countries use dinar currency
queryString = """
SELECT DISTINCT ?country ?name 
WHERE {
   ?country wdt:P31 ?class;
            wdt:P38 wd:Q191830.
   ?class wdt:P279* wd:Q6256.
   
   # get the label
   ?country sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q484195'), ('name', 'Timurid Empire')]
[('country', 'http://www.wikidata.org/entity/Q160307'), ('name', 'Fatimid Caliphate')]
2


In [28]:
#return which countries use pound sterling currency
queryString = """
SELECT DISTINCT ?country ?name 
WHERE {
   ?country wdt:P31 ?class;
            wdt:P38 wd:Q25224.
   ?class wdt:P279* wd:Q6256.
   
   # get the label
   ?country sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q145'), ('name', 'United Kingdom')]
[('country', 'http://www.wikidata.org/entity/Q25'), ('name', 'Wales')]
[('country', 'http://www.wikidata.org/entity/Q8680'), ('name', 'British Empire')]
3


In [32]:
#return population for Wales
queryString = """
SELECT ?population
WHERE {
   
   wd:Q25 wdt:P1082 ?population.
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('population', '3113000')]
1


In [33]:
#return population for British Empire
queryString = """
SELECT ?population
WHERE {
   
   wd:Q8680 wdt:P1082 ?population.
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('population', '680000000')]
1


In [34]:
#return what British Empire is instance of
queryString = """
SELECT ?obj ?name
WHERE {
   
   wd:Q8680 wdt:P31 ?obj.
   ?obj sc:name ?name.
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q3024240'), ('name', 'historical country')]
[('obj', 'http://www.wikidata.org/entity/Q48349'), ('name', 'empire')]
[('obj', 'http://www.wikidata.org/entity/Q1211369'), ('name', 'the empire on which the sun never sets')]
3


In [36]:
#return subclasses of country
queryString = """
SELECT ?s ?name
WHERE {
   ?s wdt:P279 wd:Q6256.
   ?s sc:name ?name.
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q28864179'), ('name', 'proposed country')]
[('s', 'http://www.wikidata.org/entity/Q2281140'), ('name', 'safe country of origin')]
[('s', 'http://www.wikidata.org/entity/Q188800'), ('name', 'personal union')]
[('s', 'http://www.wikidata.org/entity/Q1323642'), ('name', 'transcontinental country')]
[('s', 'http://www.wikidata.org/entity/Q132453'), ('name', 'developed country')]
[('s', 'http://www.wikidata.org/entity/Q177323'), ('name', 'developing country')]
[('s', 'http://www.wikidata.org/entity/Q5170193'), ('name', 'core country')]
[('s', 'http://www.wikidata.org/entity/Q63441421'), ('name', 'high-income country')]
[('s', 'http://www.wikidata.org/entity/Q160381'), ('name', 'Western world')]
[('s', 'http://www.wikidata.org/entity/Q112099'), ('name', 'island nation')]
[('s', 'http://www.wikidata.org/entity/Q123480'), ('name', 'landlocked country')]
[('s', 'http://www.wikidata.org/entity/Q3624078'), ('name', 'sovereign state')]
[('s', 'http://

In [4]:
#return objects linked by country property to euro
queryString = """
SELECT ?o ?name
WHERE {
   wd:Q4916 wdt:P17 ?o.
   ?o sc:name ?name.
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
[('o', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany')]
[('o', 'http://www.wikidata.org/entity/Q191'), ('name', 'Estonia')]
[('o', 'http://www.wikidata.org/entity/Q211'), ('name', 'Latvia')]
[('o', 'http://www.wikidata.org/entity/Q214'), ('name', 'Slovakia')]
[('o', 'http://www.wikidata.org/entity/Q215'), ('name', 'Slovenia')]
[('o', 'http://www.wikidata.org/entity/Q228'), ('name', 'Andorra')]
[('o', 'http://www.wikidata.org/entity/Q229'), ('name', 'Cyprus')]
[('o', 'http://www.wikidata.org/entity/Q233'), ('name', 'Malta')]
[('o', 'http://www.wikidata.org/entity/Q235'), ('name', 'Monaco')]
[('o', 'http://www.wikidata.org/entity/Q237'), ('name', 'Vatican City')]
[('o', 'http://www.wikidata.org/entity/Q238'), ('name', 'San Marino')]
[('o', 'http://www.wikidata.org/entity/Q27'), ('name', 'Ireland')]
[('o', 'http://www.wikidata.org/entity/Q29'), ('name', 'Spain')]
[('o', 'http://www.wikidata.org/entity/Q

In [7]:
# return countries currently using euro and their population
queryString = """
SELECT DISTINCT ?s ?name ?people
WHERE {
   ?s wdt:P31/wdt:P279* wd:Q6256;
      wdt:P38 wd:Q4916.
   ?s wdt:P1082 ?people.
   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q142'), ('name', 'France'), ('people', '67063703')]
[('s', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany'), ('people', '83149300')]
[('s', 'http://www.wikidata.org/entity/Q191'), ('name', 'Estonia'), ('people', '1324820')]
[('s', 'http://www.wikidata.org/entity/Q211'), ('name', 'Latvia'), ('people', '1909000')]
[('s', 'http://www.wikidata.org/entity/Q214'), ('name', 'Slovakia'), ('people', '5397036')]
[('s', 'http://www.wikidata.org/entity/Q215'), ('name', 'Slovenia'), ('people', '2066880')]
[('s', 'http://www.wikidata.org/entity/Q228'), ('name', 'Andorra'), ('people', '78151')]
[('s', 'http://www.wikidata.org/entity/Q229'), ('name', 'Cyprus'), ('people', '1141166')]
[('s', 'http://www.wikidata.org/entity/Q233'), ('name', 'Malta'), ('people', '465292')]
[('s', 'http://www.wikidata.org/entity/Q235'), ('name', 'Monaco'), ('people', '38695')]
[('s', 'http://www.wikidata.org/entity/Q236'), ('name', 'Montenegro'), ('people', '622359

In [5]:
#return objects linked by country property to pound sterling
queryString = """
SELECT ?o ?name
WHERE {
   wd:Q25224 wdt:P17 ?o.
   ?o sc:name ?name.
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q145'), ('name', 'United Kingdom')]
1


In [68]:
#final query try on updated task 6
queryString = """
SELECT ?currency ?currencyName (SUM(?people) AS ?totalPeople) WHERE {
    ?currency wdt:P17 ?country.
    ?country wdt:P31 ?class.
    ?class wdt:P279* wd:Q6256.
    ?country wdt:P1082 ?people.
    {
            SELECT DISTINCT ?currency ?currencyName
            WHERE {
               ?country wdt:P31 ?class.
               ?currency wdt:P31 wd:Q8142.
               ?currency wdt:P17 ?country.
               ?class wdt:P279* wd:Q6256.

               # get the label
               ?currency sc:name ?currencyName.
            }
    GROUP BY ?currency ?currencyName
    HAVING (COUNT(DISTINCT ?country) >= 2)
    }
}
GROUP BY ?currency ?currencyName 
ORDER BY DESC (?totalPeople)
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('currency', 'http://www.wikidata.org/entity/Q4916'), ('currencyName', 'euro'), ('totalPeople', '1545167936')]
[('currency', 'http://www.wikidata.org/entity/Q5527593'), ('currencyName', 'Gaucho'), ('totalPeople', '1033025404')]
[('currency', 'http://www.wikidata.org/entity/Q172872'), ('currencyName', 'Turkish lira'), ('totalPeople', '836770872')]
[('currency', 'http://www.wikidata.org/entity/Q861690'), ('currencyName', 'West African CFA franc'), ('totalPeople', '606963912')]
[('currency', 'http://www.wikidata.org/entity/Q184172'), ('currencyName', 'French franc'), ('totalPeople', '536899618')]
[('currency', 'http://www.wikidata.org/entity/Q1415602'), ('currencyName', 'Kopeck'), ('totalPeople', '378725400')]
[('currency', 'http://www.wikidata.org/entity/Q847739'), ('currencyName', 'Central African CFA franc'), ('totalPeople', '244664546')]
[('currency', 'http://www.wikidata.org/entity/Q3658911'), ('currencyName', 'Carlino'), ('totalPeople', '243129168')]
[('currency', 'http://w

In [112]:
#final query try
queryString = """    
 SELECT ?currency ?currencyName (SUM( ?people) AS ?totalPeople) WHERE {    
    ?country wdt:P38 ?currency.
    ?country wdt:P31 ?class.
    ?country wdt:P1082 ?people.
    FILTER(NOT EXISTS {?country wdt:P31 wd:Q3024240.} && NOT EXISTS {?country wdt:P1366 ?o.}
                    && NOT EXISTS {?country wdt:P31 wd:Q417175.} && EXISTS{?class wdt:P279* wd:Q6256.})
        {
        SELECT ?currency ?currencyName  WHERE {
           ?country wdt:P31 ?class;
                     wdt:P38 ?currency.
           ?currency wdt:P31 wd:Q8142.
           FILTER(NOT EXISTS {?country wdt:P31 wd:Q3024240.} && NOT EXISTS {?country wdt:P1366 ?o.}
                    && NOT EXISTS {?country wdt:P31 wd:Q417175.} && EXISTS{?class wdt:P279* wd:Q6256.})

           # get the label
           ?currency sc:name ?currencyName.
           ?country sc:name ?countryName.
        } 
        GROUP BY ?currency ?currencyName
        HAVING ( COUNT(DISTINCT ?country) >= 2 )
        }
    }
    GROUP BY ?currency ?currencyName
    ORDER BY DESC (?totalPeople)
"""

print("Results")
x=run_query(queryString)

Results
[('currency', 'http://www.wikidata.org/entity/Q4917'), ('currencyName', 'United States dollar'), ('totalPeople', '776682847')]
[('currency', 'http://www.wikidata.org/entity/Q4916'), ('currencyName', 'euro'), ('totalPeople', '707387342')]
[('currency', 'http://www.wikidata.org/entity/Q172872'), ('currencyName', 'Turkish lira'), ('totalPeople', '334771074')]
[('currency', 'http://www.wikidata.org/entity/Q25224'), ('currencyName', 'pound sterling'), ('totalPeople', '333224365')]
[('currency', 'http://www.wikidata.org/entity/Q861690'), ('currencyName', 'West African CFA franc'), ('totalPeople', '303481956')]
[('currency', 'http://www.wikidata.org/entity/Q41044'), ('currencyName', 'Russian ruble'), ('totalPeople', '146857904')]
[('currency', 'http://www.wikidata.org/entity/Q181907'), ('currencyName', 'South African rand'), ('totalPeople', '124312288')]
[('currency', 'http://www.wikidata.org/entity/Q847739'), ('currencyName', 'Central African CFA franc'), ('totalPeople', '122332273')

In [26]:
#add distinct on SUM
queryString = """    
 SELECT ?currency ?currencyName (SUM(DISTINCT ?people) AS ?totalPeople) WHERE {    
    ?country wdt:P38 ?currency.
    ?country wdt:P31 ?class.
    ?country wdt:P1082 ?people.
    FILTER(NOT EXISTS {?country wdt:P31 wd:Q3024240.} && NOT EXISTS {?country wdt:P1366 ?o.}
                    && NOT EXISTS {?country wdt:P31 wd:Q417175.} && EXISTS{?class wdt:P279* wd:Q6256.})
        {
        SELECT ?currency ?currencyName  WHERE {
           ?country wdt:P31 ?class;
                     wdt:P38 ?currency.
           ?currency wdt:P31 wd:Q8142.
           FILTER(NOT EXISTS {?country wdt:P31 wd:Q3024240.} && NOT EXISTS {?country wdt:P1366 ?o.}
                    && NOT EXISTS {?country wdt:P31 wd:Q417175.} && EXISTS{?class wdt:P279* wd:Q6256.})

           # get the label
           ?currency sc:name ?currencyName.
           ?country sc:name ?countryName.
        } 
        GROUP BY ?currency ?currencyName
        HAVING ( COUNT(DISTINCT ?country) >= 2 )
        }
    }
    GROUP BY ?currency ?currencyName
    ORDER BY DESC (?totalPeople)
"""

print("Results")
x=run_query(queryString)

Results
[('currency', 'http://www.wikidata.org/entity/Q4917'), ('currencyName', 'United States dollar'), ('totalPeople', '381442826')]
[('currency', 'http://www.wikidata.org/entity/Q4916'), ('currencyName', 'euro'), ('totalPeople', '342651792')]
[('currency', 'http://www.wikidata.org/entity/Q41044'), ('currencyName', 'Russian ruble'), ('totalPeople', '146857904')]
[('currency', 'http://www.wikidata.org/entity/Q861690'), ('currencyName', 'West African CFA franc'), ('totalPeople', '121487314')]
[('currency', 'http://www.wikidata.org/entity/Q172872'), ('currencyName', 'Turkish lira'), ('totalPeople', '83927988')]
[('currency', 'http://www.wikidata.org/entity/Q25224'), ('currencyName', 'pound sterling'), ('totalPeople', '69135273')]
[('currency', 'http://www.wikidata.org/entity/Q181907'), ('currencyName', 'South African rand'), ('totalPeople', '62156144')]
[('currency', 'http://www.wikidata.org/entity/Q847739'), ('currencyName', 'Central African CFA franc'), ('totalPeople', '52744134')]
[(

In [29]:
#add path instance of / country or subclass of it
queryString = """    
 SELECT ?currency ?currencyName (SUM(DISTINCT ?people) AS ?totalPeople) WHERE {    
    ?country wdt:P38 ?currency.
    ?country wdt:P31/wdt:P279* wd:Q6256.
    ?country wdt:P1082 ?people.
    FILTER(NOT EXISTS {?country wdt:P31 wd:Q3024240.} && NOT EXISTS {?country wdt:P1366 ?o.}
                    && NOT EXISTS {?country wdt:P31 wd:Q417175.})
        {
        SELECT ?currency ?currencyName  WHERE {
           ?country wdt:P31/wdt:P279* wd:Q6256;
                     wdt:P38 ?currency.
           ?currency wdt:P31 wd:Q8142.
           FILTER(NOT EXISTS {?country wdt:P31 wd:Q3024240.} && NOT EXISTS {?country wdt:P1366 ?o.}
                    && NOT EXISTS {?country wdt:P31 wd:Q417175.})

           # get the label
           ?currency sc:name ?currencyName.
           ?country sc:name ?countryName.
        } 
        GROUP BY ?currency ?currencyName
        HAVING ( COUNT(DISTINCT ?country) >= 2 )
        }
    }
    GROUP BY ?currency ?currencyName
    ORDER BY DESC (?totalPeople)
"""

print("Results")
x=run_query(queryString)

Results
[('currency', 'http://www.wikidata.org/entity/Q4917'), ('currencyName', 'United States dollar'), ('totalPeople', '381442826')]
[('currency', 'http://www.wikidata.org/entity/Q4916'), ('currencyName', 'euro'), ('totalPeople', '342651792')]
[('currency', 'http://www.wikidata.org/entity/Q41044'), ('currencyName', 'Russian ruble'), ('totalPeople', '146857904')]
[('currency', 'http://www.wikidata.org/entity/Q861690'), ('currencyName', 'West African CFA franc'), ('totalPeople', '121487314')]
[('currency', 'http://www.wikidata.org/entity/Q172872'), ('currencyName', 'Turkish lira'), ('totalPeople', '83927988')]
[('currency', 'http://www.wikidata.org/entity/Q25224'), ('currencyName', 'pound sterling'), ('totalPeople', '69135273')]
[('currency', 'http://www.wikidata.org/entity/Q181907'), ('currencyName', 'South African rand'), ('totalPeople', '62156144')]
[('currency', 'http://www.wikidata.org/entity/Q847739'), ('currencyName', 'Central African CFA franc'), ('totalPeople', '52744134')]
[(

In [ ]:
#return population for United Kingdom and check the sum with the Wales' one
queryString = """
SELECT ?population
WHERE {
   
   wd:Q145 wdt:P1082 ?population.
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('population', '66022273')]
1


In [2]:
#subquery don't needed
queryString = """    
SELECT ?currency ?currencyName (SUM(DISTINCT ?people) AS ?totalPeople) (COUNT(DISTINCT ?country) as ?n)  WHERE {    
    ?country wdt:P38 ?currency.
    ?country wdt:P31/wdt:P279* wd:Q6256.
    ?country wdt:P1082 ?people.
    ?currency wdt:P31 wd:Q8142.
    FILTER(NOT EXISTS {?country wdt:P31 wd:Q3024240.} && NOT EXISTS {?country wdt:P1366 ?o.}
                    && NOT EXISTS {?country wdt:P31 wd:Q417175.})
    ?currency sc:name ?currencyName.
    ?country sc:name ?countryName.    
}       
GROUP BY ?currency ?currencyName
HAVING (COUNT(DISTINCT ?country) >= 2)
ORDER BY DESC (?totalPeople)
"""

print("Results")
x=run_query(queryString)

Results
[('currency', 'http://www.wikidata.org/entity/Q4916'), ('currencyName', 'euro'), ('totalPeople', '1102614348'), ('n', '24')]
[('currency', 'http://www.wikidata.org/entity/Q4917'), ('currencyName', 'United States dollar'), ('totalPeople', '380364982'), ('n', '15')]
[('currency', 'http://www.wikidata.org/entity/Q209272'), ('currencyName', 'Uruguayan peso'), ('totalPeople', '267804570'), ('n', '2')]
[('currency', 'http://www.wikidata.org/entity/Q8146'), ('currencyName', 'Japanese yen'), ('totalPeople', '148744875'), ('n', '2')]
[('currency', 'http://www.wikidata.org/entity/Q861690'), ('currencyName', 'West African CFA franc'), ('totalPeople', '121487314'), ('n', '8')]
[('currency', 'http://www.wikidata.org/entity/Q199462'), ('currencyName', 'Egyptian pound'), ('totalPeople', '97798827'), ('n', '2')]
[('currency', 'http://www.wikidata.org/entity/Q188608'), ('currencyName', 'Iranian rial'), ('totalPeople', '88813012'), ('n', '2')]
[('currency', 'http://www.wikidata.org/entity/Q17287

In [11]:
#pass the country to the outer query
queryString = """    
SELECT ?currency ?currencyName (SUM(DISTINCT ?people) AS ?totalPeople) (COUNT(DISTINCT ?country)as ?n) (GROUP_CONCAT(DISTINCT ?countryName ; separator =", " ) AS ?list) WHERE {    
    ?country wdt:P1082 ?people.
    {
        SELECT ?country ?countryName ?currency ?currencyName WHERE {    
            ?country wdt:P38 ?currency.
            ?country wdt:P31/wdt:P279* wd:Q6256.

            ?currency wdt:P31 wd:Q8142.
            FILTER(NOT EXISTS {?country wdt:P31 wd:Q3024240.} && NOT EXISTS {?country wdt:P1366 ?o.}
                            && NOT EXISTS {?country wdt:P31 wd:Q417175.})
            ?currency sc:name ?currencyName.
            ?country sc:name ?countryName.    
        }
    }
}
GROUP BY ?currency ?currencyName
HAVING (COUNT(DISTINCT ?country) >= 2)
ORDER BY DESC (?totalPeople)
"""

print("Results")
x=run_query(queryString)

Results
[('currency', 'http://www.wikidata.org/entity/Q4917'), ('currencyName', 'United States dollar'), ('totalPeople', '381442826'), ('n', '12'), ('list', 'American Samoa, East Timor, Ecuador, El Salvador, Federated States of Micronesia, Liberia, Marshall Islands, Northern Mariana Islands, Palau, Panama, United States of America, Zimbabwe')]
[('currency', 'http://www.wikidata.org/entity/Q4916'), ('currencyName', 'euro'), ('totalPeople', '342651792'), ('n', '24'), ('list', 'Andorra, Austria, Belgium, Cyprus, Estonia, Finland, France, Germany, Greece, Ireland, Italy, Latvia, Lithuania, Luxembourg, Malta, Monaco, Montenegro, Netherlands, Portugal, San Marino, Slovakia, Slovenia, Spain, Vatican City')]
[('currency', 'http://www.wikidata.org/entity/Q41044'), ('currencyName', 'Russian ruble'), ('totalPeople', '146857904'), ('n', '2'), ('list', 'Russia, South Ossetia')]
[('currency', 'http://www.wikidata.org/entity/Q861690'), ('currencyName', 'West African CFA franc'), ('totalPeople', '1214

Final query for this task

In [12]:
#Consider the currencies obtained in task 6, for each currency return the number of people using that currency 
#(hint: consider the maximum popolation available for each country).
# write your final query
queryString = """    
SELECT ?currency ?currencyName (SUM(DISTINCT ?people) AS ?totalPeople) WHERE {    
    ?country wdt:P1082 ?people.
    {
        SELECT ?country ?currency ?currencyName WHERE {    
            ?country wdt:P38 ?currency.
            ?country wdt:P31/wdt:P279* wd:Q6256.
            ?currency wdt:P31 wd:Q8142.
            FILTER(NOT EXISTS {?country wdt:P31 wd:Q3024240.} && NOT EXISTS {?country wdt:P1366 ?o.}
                            && NOT EXISTS {?country wdt:P31 wd:Q417175.})
            ?currency sc:name ?currencyName.   
        }
    }
}
GROUP BY ?currency ?currencyName
HAVING (COUNT(DISTINCT ?country) >= 2)
ORDER BY DESC (?totalPeople)
"""

print("Results")
x=run_query(queryString)

Results
[('currency', 'http://www.wikidata.org/entity/Q4917'), ('currencyName', 'United States dollar'), ('totalPeople', '381442826')]
[('currency', 'http://www.wikidata.org/entity/Q4916'), ('currencyName', 'euro'), ('totalPeople', '342651792')]
[('currency', 'http://www.wikidata.org/entity/Q41044'), ('currencyName', 'Russian ruble'), ('totalPeople', '146857904')]
[('currency', 'http://www.wikidata.org/entity/Q861690'), ('currencyName', 'West African CFA franc'), ('totalPeople', '121487314')]
[('currency', 'http://www.wikidata.org/entity/Q172872'), ('currencyName', 'Turkish lira'), ('totalPeople', '83927988')]
[('currency', 'http://www.wikidata.org/entity/Q25224'), ('currencyName', 'pound sterling'), ('totalPeople', '69135273')]
[('currency', 'http://www.wikidata.org/entity/Q181907'), ('currencyName', 'South African rand'), ('totalPeople', '62156144')]
[('currency', 'http://www.wikidata.org/entity/Q847739'), ('currencyName', 'Central African CFA franc'), ('totalPeople', '52744134')]
[(